# 0. PopForecast — Modeling & Robustness

**Purpose**

- Operationalize the Cycle 3 pivot: combine tree-based non-linearity with robustness to outliers.
- Compare XGBoost (naive vs robust objectives) against the frozen baseline (HuberRegressor) on the 2021 test set.
- Decide whether robust XGBoost beats MAE 15.21 and, if so, persist the best model to `models/cycle_03/xgb_robust.joblib`.

# 1. Setup

## 1.1 - Project root & module path setup

In [335]:
from __future__ import annotations

import sys
from pathlib import Path
from typing import Final

# --- Project root setup (so `src/` is importable from notebooks/) ---
PROJECT_ROOT: Final[Path] = Path.cwd().parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("Project root:", PROJECT_ROOT)

Project root: /mnt/c/Users/Daniel/OneDrive/Documentos/_Cursos/Outros/PopForecast


## 1.2 - Project paths

In [336]:
# --- Data input ---
DATA_PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "spotify_tracks_modeling.parquet"

# --- Cycle 2 (frozen config as single source of truth) ---
CYCLE2_MODELS_DIR = PROJECT_ROOT / "models" / "cycle_02"
FROZEN_CONFIG_PATH = CYCLE2_MODELS_DIR / "baseline_huber15_audit_v3_from_pack.json"

# --- Cycle 3 (outputs; no mkdir at setup time) ---
CYCLE3_MODELS_DIR = PROJECT_ROOT / "models" / "cycle_03"
BEST_MODEL_PATH = CYCLE3_MODELS_DIR / "xgb_robust.joblib"
CHAMPION_PATH = CYCLE3_MODELS_DIR / "champion.joblib"
RUN_METADATA_PATH = CYCLE3_MODELS_DIR / "run_metadata_cycle3.json"


print("Processed dataset:", DATA_PROCESSED_PATH)
print("Frozen config:", FROZEN_CONFIG_PATH)
print("Cycle 3 models dir:", CYCLE3_MODELS_DIR)
print("Best model path:", BEST_MODEL_PATH)

if not DATA_PROCESSED_PATH.exists():
    raise FileNotFoundError(f"Processed dataset not found: {DATA_PROCESSED_PATH}")

if not FROZEN_CONFIG_PATH.exists():
    raise FileNotFoundError(f"Frozen config not found: {FROZEN_CONFIG_PATH}")


Processed dataset: /mnt/c/Users/Daniel/OneDrive/Documentos/_Cursos/Outros/PopForecast/data/processed/spotify_tracks_modeling.parquet
Frozen config: /mnt/c/Users/Daniel/OneDrive/Documentos/_Cursos/Outros/PopForecast/models/cycle_02/baseline_huber15_audit_v3_from_pack.json
Cycle 3 models dir: /mnt/c/Users/Daniel/OneDrive/Documentos/_Cursos/Outros/PopForecast/models/cycle_03
Best model path: /mnt/c/Users/Daniel/OneDrive/Documentos/_Cursos/Outros/PopForecast/models/cycle_03/xgb_robust.joblib


## 1.3 - Imports

In [292]:
# --- Standard library ---
import gc
import json
from dataclasses import asdict
from typing import Any, Dict

import hashlib
import json
from pathlib import Path
from typing import Any, Dict, Union

# --- Third-party ---
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import xgboost as xgb
from xgboost import XGBRegressor

# --- Scikit-learn ---
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

# --- Local project (src/) ---
from src.core.features import (
    FeatureEngineeringConfig,
    apply_feature_engineering,
    build_feature_pipeline,
)
from src.core.preprocessing import default_config, run_preprocessing



from __future__ import annotations

# --- Standard library ---
import hashlib
import json
from pathlib import Path
from typing import Any

from __future__ import annotations

# --- Standard library ---
import hashlib
import json
from pathlib import Path
from typing import Any

from __future__ import annotations

from dataclasses import dataclass
from typing import Dict, Tuple

import numpy as np
import sklearn
import xgboost as xgb
from xgboost import XGBRegressor

from __future__ import annotations

from typing import Any

import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from typing import Iterable
from typing import Literal


from __future__ import annotations

import numpy as np
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_absolute_error

from __future__ import annotations

import hashlib
import json
from pathlib import Path
from typing import Any

from __future__ import annotations

from dataclasses import dataclass
from typing import Any, Optional

import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

from __future__ import annotations

from dataclasses import asdict
from typing import Any

import numpy as np
import pandas as pd
from scipy.stats import randint, uniform
from xgboost import XGBRegressor

from sklearn.model_selection import PredefinedSplit, RandomizedSearchCV

from __future__ import annotations

from dataclasses import dataclass
from typing import Any

import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import PredefinedSplit, RandomizedSearchCV
import numpy as np
from sklearn.linear_model import LogisticRegression, HuberRegressor
from sklearn.metrics import mean_absolute_error

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRegressor


from dataclasses import dataclass
from typing import Any
import numpy as np

from sklearn.model_selection import ParameterSampler, PredefinedSplit
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRegressor
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import ParameterSampler

import numpy as np
from sklearn.linear_model import LogisticRegression, HuberRegressor
from sklearn.metrics import mean_absolute_error


from joblib import Parallel, delayed
import numpy as np
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor



from joblib import Parallel, delayed
import numpy as np
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor


import numpy as np
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_absolute_error

from pathlib import Path
import json
import joblib
import platform
import sys
import hashlib

import numpy as np
import pandas as pd
import sklearn
import xgboost as xgb


from pathlib import Path
import joblib

## 1.4 - Global settings

In [4]:
# --- Reproducibility (only for stochastic procedures inside this notebook) ---
RANDOM_SEED = 42

# --- Pandas display ---
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)
pd.set_option("display.max_colwidth", 60)
pd.set_option("display.float_format", "{:,.4f}".format)

# --- Matplotlib defaults (lightweight) ---
plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.grid"] = True

## 1.5 - Support functions

In [293]:
def _sha256_of_bytes(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()


def _extract_cycle3_protocol_key(config: dict[str, Any]) -> str:
    """
    Extract the Cycle 3 frozen protocol key from the config.

    Supported schemas:
    1) frozen_track_cycle3: "<protocol_key>"
    2) frozen_track_cycle3: {"protocol_name": "<protocol_key>", ...}
       (also accepts: "protocol", "protocol_key", "key", "name")
    3) frozen_track_cycle3: {...} without protocol key:
       - If baseline_protocols has exactly one entry, that entry is used as fallback.
       - If multiple entries exist, raise (ambiguous).
    """
    baseline_protocols = config.get("baseline_protocols")
    if not isinstance(baseline_protocols, dict) or not baseline_protocols:
        raise ValueError("'baseline_protocols' must be a non-empty dict.")

    raw = config.get("frozen_track_cycle3")

    # Case 1: string directly names the protocol
    if isinstance(raw, str) and raw.strip():
        return raw.strip()

    # Case 2: dict includes protocol key
    if isinstance(raw, dict):
        for candidate_key in ("protocol_name", "protocol", "protocol_key", "key", "name"):
            val = raw.get(candidate_key)
            if isinstance(val, str) and val.strip():
                return val.strip()

    # Case 3: fallback if exactly one protocol exists
    if len(baseline_protocols) == 1:
        return next(iter(baseline_protocols.keys()))

    raise ValueError(
        "Could not determine the Cycle 3 frozen protocol key. "
        "Either set 'frozen_track_cycle3' as a non-empty string protocol key, "
        "or include one of: ['protocol_name','protocol','protocol_key','key','name'], "
        "or keep exactly one entry in 'baseline_protocols'."
    )


def load_frozen_config(path: str | Path) -> dict[str, Any]:
    """
    Load the official frozen Cycle 2 configuration JSON used as the single source of truth for Cycle 3.

    - Returns a plain dict to keep schema flexible.
    - Validates the presence of core top-level keys.
    - Extracts the Cycle 3 frozen protocol key in a schema-tolerant way.
    - Prints a SHA256 digest for traceability.
    """
    config_path = Path(path)

    if not config_path.exists():
        raise FileNotFoundError(f"Frozen config not found: {config_path}")

    raw_bytes = config_path.read_bytes()
    digest = _sha256_of_bytes(raw_bytes)

    config = json.loads(raw_bytes.decode("utf-8"))

    if not isinstance(config, dict) or not config:
        raise ValueError("Frozen config must be a non-empty JSON object (dict).")

    required_top_level_keys = (
        "cycle",
        "description",
        "decision_split",
        "guardrail_split",
        "metrics",
        "baseline_protocols",
        "frozen_track_cycle3",
    )
    missing = [k for k in required_top_level_keys if k not in config]
    if missing:
        raise KeyError(f"Frozen config is missing required keys: {missing}")

    protocol_key = _extract_cycle3_protocol_key(config)

    if protocol_key not in config["baseline_protocols"]:
        raise KeyError(
            "Extracted Cycle 3 protocol key is not present in 'baseline_protocols'. "
            f"Missing protocol key: {protocol_key}"
        )

    print(f"Frozen config loaded: {config_path.name}")
    print(f"Frozen config SHA256: {digest}")
    print(f"Cycle 3 frozen protocol: {protocol_key}")

    return config



def _to_1d_float_array(x: np.ndarray | pd.Series | list, *, name: str) -> np.ndarray:
    arr = np.asarray(x, dtype=float)

    if arr.ndim == 0:
        raise ValueError(f"{name} must be 1D-like, got a scalar.")
    if arr.ndim > 1:
        arr = arr.reshape(-1)

    return arr


def regression_metrics(
    y_true: np.ndarray,
    y_pred: np.ndarray,
) -> dict[str, float]:
    """Global regression metrics (MAE as primary)."""
    mae = mean_absolute_error(y_true, y_pred)
    return {"mae": float(mae)}


def segmented_metrics(
    y_true: np.ndarray,
    y_pred: np.ndarray,
) -> dict[str, float]:
    """
    Segment metrics by whether the true target is zero.

    This is useful under zero-inflation drift (e.g., 2021 jump).
    """
    y_true_arr = np.asarray(y_true, dtype=float).reshape(-1)
    y_pred_arr = np.asarray(y_pred, dtype=float).reshape(-1)

    zero_mask = y_true_arr == 0
    pos_mask = ~zero_mask

    out: dict[str, float] = {
        "zero_rate_true": float(zero_mask.mean()),
        "pos_rate_true": float(pos_mask.mean()),
    }

    if zero_mask.any():
        out["mae_zero"] = float(mean_absolute_error(y_true_arr[zero_mask], y_pred_arr[zero_mask]))
    else:
        out["mae_zero"] = float("nan")

    if pos_mask.any():
        out["mae_pos"] = float(mean_absolute_error(y_true_arr[pos_mask], y_pred_arr[pos_mask]))
    else:
        out["mae_pos"] = float("nan")

    return out


def full_metrics(
    y_true: np.ndarray | pd.Series | list,
    y_pred: np.ndarray | pd.Series | list,
) -> dict[str, float]:
    """Convenience wrapper: global + segmented metrics."""
    y_true_arr = _to_1d_float_array(y_true, name="y_true")
    y_pred_arr = _to_1d_float_array(y_pred, name="y_pred")

    if y_true_arr.shape[0] != y_pred_arr.shape[0]:
        raise ValueError(
            "y_true and y_pred must have the same length. "
            f"Got {y_true_arr.shape[0]} and {y_pred_arr.shape[0]}."
        )

    out: dict[str, float] = {}
    out.update(regression_metrics(y_true_arr, y_pred_arr))
    out.update(segmented_metrics(y_true_arr, y_pred_arr))
    return out


def split_table(
    df: pd.DataFrame,
    split_masks: dict[str, pd.Series],
    *,
    year_col: str,
    target_col: str | None = None,
) -> pd.DataFrame:
    """
    Build a compact summary table for each split (rows, year range, and optional target stats).

    Parameters
    ----------
    df:
        Full dataframe containing at least `year_col` and (optionally) `target_col`.
    split_masks:
        Mapping like {"train": mask, "val": mask, "test": mask}, where each mask is a boolean Series.
    year_col:
        Column holding the release year used for temporal splits.
    target_col:
        If provided, computes basic target stats (mean/median and zero rate).

    Returns
    -------
    pd.DataFrame
        One row per split.
    """
    rows: list[dict[str, object]] = []

    for split_name, mask in split_masks.items():
        split_df = df.loc[mask]
        years = split_df[year_col].dropna()

        summary: dict[str, object] = {
            "split": split_name,
            "rows": int(split_df.shape[0]),
            "min_year": int(years.min()) if not years.empty else None,
            "max_year": int(years.max()) if not years.empty else None,
        }

        if target_col is not None:
            y = pd.to_numeric(split_df[target_col], errors="coerce")
            summary["target_mean"] = float(y.mean()) if y.notna().any() else None
            summary["target_median"] = float(y.median()) if y.notna().any() else None
            summary["target_zero_rate"] = float((y == 0).mean()) if y.notna().any() else None

        rows.append(summary)

    return pd.DataFrame(rows).sort_values("split").reset_index(drop=True)


def assert_expected_year_coverage(
    df: pd.DataFrame,
    split_masks: dict[str, pd.Series],
    *,
    year_col: str,
    train_max_year: int,
    val_years: set[int],
    test_years: set[int],
) -> None:
    """
    Fail fast if the temporal split does not match the frozen expectations.
    """
    required_splits = {"train", "val", "test"}
    missing = required_splits.difference(split_masks.keys())
    if missing:
        raise KeyError(f"Missing required split masks: {sorted(missing)}")

    def _years_for(mask: pd.Series) -> set[int]:
        years = pd.to_numeric(df.loc[mask, year_col], errors="coerce").dropna().astype(int)
        return set(years.tolist())

    train_years = _years_for(split_masks["train"])
    val_years_found = _years_for(split_masks["val"])
    test_years_found = _years_for(split_masks["test"])

    if df.loc[split_masks["train"]].empty:
        raise AssertionError("Train split is empty.")
    if df.loc[split_masks["val"]].empty:
        raise AssertionError("Validation split is empty.")
    if df.loc[split_masks["test"]].empty:
        raise AssertionError("Test split is empty.")

    if any(y > train_max_year for y in train_years):
        raise AssertionError(
            f"Train split contains years > {train_max_year}. Found: {sorted(train_years)}"
        )

    if val_years_found != val_years:
        raise AssertionError(
            f"Validation years mismatch. Expected: {sorted(val_years)}; Found: {sorted(val_years_found)}"
        )

    if test_years_found != test_years:
        raise AssertionError(
            f"Test years mismatch. Expected: {sorted(test_years)}; Found: {sorted(test_years_found)}"
        )


@dataclass(frozen=True)
class XgbObjectiveCheckResult:
    ok: bool
    error: str


def check_xgboost_objectives(
    objectives: Tuple[str, ...],
    random_seed: int = 42,
) -> Dict[str, XgbObjectiveCheckResult]:
    """
    Smoke-test XGBoost objectives by fitting a tiny model.
    This verifies that the installed XGBoost build supports the objectives at runtime.
    """
    rng = np.random.default_rng(random_seed)
    x_train = rng.normal(size=(64, 4)).astype(np.float32)
    y_train = rng.normal(size=(64,)).astype(np.float32)

    results: Dict[str, XgbObjectiveCheckResult] = {}

    for obj in objectives:
        try:
            model = XGBRegressor(
                objective=obj,
                n_estimators=5,
                max_depth=3,
                learning_rate=0.2,
                subsample=0.9,
                colsample_bytree=0.9,
                random_state=RANDOM_SEED,
                n_jobs=1,
                tree_method="hist",
                verbosity=0,
            )
            model.fit(x_train, y_train)
            results[obj] = XgbObjectiveCheckResult(ok=True, error="")
        except Exception as exc:  # noqa: BLE001
            results[obj] = XgbObjectiveCheckResult(ok=False, error=str(exc))

    return results


from __future__ import annotations

from typing import Literal

import pandas as pd


def build_temporal_split_masks(
    df: pd.DataFrame,
    *,
    year_col: str,
    train_max_year: int,
    val_year: int,
    test_year: int,
    nan_policy: Literal["error", "train", "drop"] = "error",
) -> tuple[pd.Series, pd.Series, pd.Series]:
    """
    Build mutually exclusive temporal split masks.

    Parameters
    ----------
    nan_policy:
        - "error": raise if year_col contains NaNs after coercion (strict).
        - "train": assign NaN years to the train split (recommended for Huber-15 baseline parity).
        - "drop": exclude NaN years from all splits (changes population; use only if explicitly desired).
    """
    years = pd.to_numeric(df[year_col], errors="coerce")

    n_bad = int(years.isna().sum())
    if n_bad > 0 and nan_policy == "error":
        raise ValueError(f"NaN years detected in column '{year_col}': {n_bad} rows.")

    train_mask = years <= train_max_year
    val_mask = years == val_year
    test_mask = years == test_year

    if n_bad > 0 and nan_policy == "train":
        train_mask = train_mask | years.isna()
    elif n_bad > 0 and nan_policy == "drop":
        # NaNs remain excluded from all masks (default behavior)
        pass

    if not (train_mask.any() and val_mask.any() and test_mask.any()):
        raise ValueError(
            "Split masks are empty for at least one split. "
            f"Counts: train={int(train_mask.sum())}, val={int(val_mask.sum())}, test={int(test_mask.sum())}."
        )

    overlap = (train_mask & val_mask) | (train_mask & test_mask) | (val_mask & test_mask)
    if overlap.any():
        raise ValueError("Split masks overlap. Temporal split must be mutually exclusive.")

    return train_mask, val_mask, test_mask



def extract_protocol_columns(
    frozen_config: dict[str, Any],
    *,
    protocol_key: str,
) -> list[str]:
    protocols = frozen_config.get("baseline_protocols", {})
    if protocol_key not in protocols:
        raise KeyError(f"Protocol '{protocol_key}' not found under 'baseline_protocols'.")

    protocol = protocols[protocol_key]
    cols = protocol.get("numeric_cols", [])
    if not isinstance(cols, list) or not cols:
        raise ValueError(f"Protocol '{protocol_key}' has no valid 'numeric_cols' list.")
    return [str(c) for c in cols]


def fit_train_only_median_imputer(
    X_train: pd.DataFrame,
) -> SimpleImputer:
    imputer = SimpleImputer(strategy="median")
    imputer.fit(X_train)
    return imputer


def transform_with_imputer(
    imputer: SimpleImputer,
    X: pd.DataFrame,
    *,
    columns: list[str],
    index: pd.Index,
) -> pd.DataFrame:
    arr = imputer.transform(X)
    return pd.DataFrame(arr, columns=columns, index=index)


def make_recency_weights(
    years: pd.Series,
    *,
    current_year: int,
    lambda_recency: float,
) -> np.ndarray:
    """
    Compute recency weights: w = exp(-lambda * age), with age = clip(current_year - year, lower=0).
    """
    year_values = pd.to_numeric(years, errors="coerce").to_numpy(dtype=float)

    if np.isnan(year_values).any():
        n_bad = int(np.isnan(year_values).sum())
        raise ValueError(f"NaN years detected while building recency weights: {n_bad} rows.")

    age = np.clip(current_year - year_values, a_min=0.0, a_max=None)
    weights = np.exp(-lambda_recency * age)
    return weights.astype(float)


def compute_recency_weights(
    years: pd.Series,
    *,
    current_year: int,
    lambda_recency: float,
) -> np.ndarray:
    """
    Compute recency weights w = exp(-lambda * age), where age = clip(current_year - year, lower=0).

    Missing/invalid years are treated as age=0 (w=1.0).
    """
    y = pd.to_numeric(years, errors="coerce").to_numpy(dtype=float)
    y = np.nan_to_num(y, nan=float(current_year))  # NaN -> current_year => age=0
    age = np.clip(float(current_year) - y, a_min=0.0, a_max=None)
    w = np.exp(-float(lambda_recency) * age)
    return w.astype(np.float64, copy=False)

    

def summarize_array(x: np.ndarray, *, name: str) -> None:
    p = np.percentile(x, [0, 1, 5, 50, 95, 99, 100])
    print(f"{name}:")
    print(f"  n={x.size}")
    print(f"  min={p[0]:.6f}  p01={p[1]:.6f}  p05={p[2]:.6f}  p50={p[3]:.6f}  p95={p[4]:.6f}  p99={p[5]:.6f}  max={p[6]:.6f}")
    print(f"  mean={x.mean():.6f}  std={x.std():.6f}")


def evaluate_mae(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(mean_absolute_error(y_true, y_pred))




def _sha256_of_bytes(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()


def load_json_dict(path: str | Path) -> dict[str, Any]:
    """Load a JSON file as a dict and print a SHA256 digest for traceability."""
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"JSON not found: {p}")

    raw = p.read_bytes()
    digest = _sha256_of_bytes(raw)

    obj = json.loads(raw.decode("utf-8"))
    if not isinstance(obj, dict) or not obj:
        raise ValueError(f"JSON must be a non-empty dict: {p}")

    print(f"Loaded: {p.name}")
    print(f"SHA256:  {digest}")
    return obj


def _require_dict_key(obj: dict[str, Any], key: str) -> dict[str, Any]:
    val = obj.get(key)
    if not isinstance(val, dict) or not val:
        raise KeyError(f"Missing or invalid dict key '{key}'.")
    return val


def summarize_governance_config(cfg: dict[str, Any]) -> None:
    """Minimal summary for the governance (Cycle 2 frozen config)."""
    print("\n[Governance config]")
    print("Top-level keys:", sorted(cfg.keys()))
    print("Cycle:", cfg.get("cycle"))
    print("Description:", cfg.get("description"))

    decision = cfg.get("decision_split", {})
    guardrail = cfg.get("guardrail_split", {})
    metrics = cfg.get("metrics", [])

    if isinstance(decision, dict):
        print("\nDecision split:")
        for k, v in decision.items():
            print(f"  - {k}: {v}")

    if isinstance(guardrail, dict):
        print("\nGuardrail split:")
        for k, v in guardrail.items():
            print(f"  - {k}: {v}")

    if isinstance(metrics, list):
        print("\nMetrics (reporting):")
        print("  -", metrics)


def summarize_baseline_audit_v3(audit: dict[str, Any]) -> None:
    """Minimal summary for baseline audit v3 (derived from the NPZ pack arrays)."""
    print("\n[Baseline audit v3]")
    print("Top-level keys:", sorted(audit.keys()))

    protocol = _require_dict_key(audit, "baseline_protocol")

    print("\nBaseline protocol:")
    print("  - name:", protocol.get("name"))
    cols = protocol.get("numeric_cols_15", [])
    if isinstance(cols, list):
        print("  - numeric_cols_15:", len(cols))
        print("  - first 15 cols:", cols[:15])

    print("\nRecency weighting:")
    print("  - recency_lambda:", protocol.get("recency_lambda"))
    print("  - current_year:", protocol.get("current_year"))

    imputer = protocol.get("imputer", {})
    if isinstance(imputer, dict):
        print("\nImputer:")
        for k, v in imputer.items():
            print(f"  - {k}: {v}")

    print("\nPack NPZ SHA256:", audit.get("pack_npz_sha256"))


def validate_cycle3_alignment(
    *,
    governance_cfg: dict[str, Any],
    baseline_audit: dict[str, Any],
) -> None:
    """
    Validate that governance config and baseline audit v3 agree on Cycle 3 invariants.

    - Governance must define the temporal decision split and include MAE in reporting metrics.
    - Audit v3 must define the baseline protocol name, the exact 15 numeric columns,
      and recency settings (lambda/current_year).
    """
    # Governance checks
    decision = governance_cfg.get("decision_split")
    if not isinstance(decision, dict):
        raise KeyError("Governance config missing 'decision_split' (dict).")

    required_decision_keys = {"train", "val", "test"}
    if not required_decision_keys.issubset(decision.keys()):
        missing = sorted(required_decision_keys - set(decision.keys()))
        raise KeyError(f"Governance decision_split missing keys: {missing}")

    metrics = governance_cfg.get("metrics", [])
    if not isinstance(metrics, list) or "mae" not in metrics:
        raise ValueError("Governance metrics must include 'mae'.")

    # Audit v3 checks
    protocol = _require_dict_key(baseline_audit, "baseline_protocol")

    protocol_name = protocol.get("name")
    if not isinstance(protocol_name, str) or not protocol_name.strip():
        raise ValueError("baseline_protocol.name must be a non-empty string.")

    cols = protocol.get("numeric_cols_15")
    if not isinstance(cols, list) or len(cols) != 15:
        raise ValueError("baseline_protocol.numeric_cols_15 must be a list of length 15.")

    recency_lambda = protocol.get("recency_lambda")
    current_year = protocol.get("current_year")
    if not isinstance(recency_lambda, (int, float)) or float(recency_lambda) <= 0.0:
        raise ValueError("baseline_protocol.recency_lambda must be a positive float.")
    if not isinstance(current_year, int):
        raise ValueError("baseline_protocol.current_year must be an int.")

    imputer = protocol.get("imputer")
    if not isinstance(imputer, dict):
        raise KeyError("baseline_protocol.imputer must be a dict.")
    if imputer.get("strategy") != "median":
        raise ValueError("baseline_protocol.imputer.strategy must be 'median'.")
    if imputer.get("fit_scope") != "train_only":
        raise ValueError("baseline_protocol.imputer.fit_scope must be 'train_only'.")

    print("\n✅ Cycle 3 alignment check passed (governance ↔ baseline audit v3).")
    print(f"   - baseline protocol: {protocol_name}")
    print(f"   - numeric cols: {len(cols)}")
    print(f"   - recency_lambda: {float(recency_lambda):.4f} | current_year: {current_year}")




@dataclass(frozen=True)
class ModelRunResult:
    model_name: str
    params: dict[str, Any]
    objective: Optional[str]
    used_sample_weight: bool
    mae_val_2020: float
    mae_test_2021: float
    pred_min_test: float
    pred_max_test: float

    def to_dict(self) -> dict[str, Any]:
        return {
            "model_name": self.model_name,
            "objective": self.objective,
            "used_sample_weight": self.used_sample_weight,
            "mae_val_2020": self.mae_val_2020,
            "mae_test_2021": self.mae_test_2021,
            "pred_min_test": self.pred_min_test,
            "pred_max_test": self.pred_max_test,
            "params": self.params,
        }


def _safe_fit(
    model: Any,
    X: pd.DataFrame,
    y: pd.Series,
    sample_weight: Optional[np.ndarray],
) -> tuple[Any, bool]:
    """
    Fit with sample_weight when supported; otherwise fall back to unweighted fit.
    Returns (fitted_model, used_sample_weight).
    """
    if sample_weight is None:
        model.fit(X, y)
        return model, False

    try:
        model.fit(X, y, sample_weight=sample_weight)
        return model, True
    except TypeError:
        model.fit(X, y)
        return model, False


def _extract_objective(model: Any) -> Optional[str]:
    """Best-effort extraction of XGBoost objective (if present)."""
    params = getattr(model, "get_params", None)
    if callable(params):
        p = model.get_params()
        obj = p.get("objective")
        return str(obj) if obj is not None else None
    return None


def run_and_evaluate_model(
    *,
    model: Any,
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_val: pd.DataFrame,
    y_val: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    sample_weight_train: Optional[np.ndarray] = None,
) -> ModelRunResult:
    """
    Protocol-guarded runner:
    - fit on train (optionally weighted),
    - evaluate MAE on Val 2020 and Test 2021,
    - return a compact, loggable result.
    """
    fitted, used_w = _safe_fit(model, X_train, y_train, sample_weight_train)

    y_pred_val = fitted.predict(X_val)
    y_pred_test = fitted.predict(X_test)

    mae_val = float(mean_absolute_error(y_val, y_pred_val))
    mae_test = float(mean_absolute_error(y_test, y_pred_test))

    pred_min = float(np.min(y_pred_test))
    pred_max = float(np.max(y_pred_test))

    model_name = type(fitted).__name__
    params = fitted.get_params() if hasattr(fitted, "get_params") else {}
    objective = _extract_objective(fitted)

    return ModelRunResult(
        model_name=model_name,
        params=params,
        objective=objective,
        used_sample_weight=used_w,
        mae_val_2020=mae_val,
        mae_test_2021=mae_test,
        pred_min_test=pred_min,
        pred_max_test=pred_max,
    )






@dataclass(frozen=True)
class TuningResult:
    best_params: dict[str, Any]
    best_val_mae: float
    best_model_eval: dict[str, Any]


def tune_xgb_with_predefinedsplit_holdout(
    *,
    objective: str,
    X_train: pd.DataFrame,
    y_train: pd.Series,
    w_train: np.ndarray,
    X_val: pd.DataFrame,
    y_val: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    base_params: dict[str, Any] | None = None,
    n_iter: int = 20,
    random_state: int = RANDOM_SEED,
    n_jobs: int = -1,
) -> TuningResult:
    """
    Tune XGBRegressor hyperparameters using a fixed holdout (Val 2020) via PredefinedSplit,
    then evaluate the best model on Val 2020 and Test 2021 using the existing harness
    `run_and_evaluate_model(...)`.

    Notes:
    - Training uses sample weights only on the TRAIN fold.
    - Val is used strictly for selection; Test is only reported after selection.
    """
    if base_params is None:
        base_params = {}

    # --- Build a single dataset for RandomizedSearchCV ---
    X_trv = pd.concat([X_train, X_val], axis=0)
    y_trv = pd.concat([y_train, y_val], axis=0)

    # weights: train weights + val weights = 1.0 (so they don't affect fitting on val fold)
    w_trv = np.concatenate([w_train, np.ones(len(y_val), dtype=float)])

    # PredefinedSplit: -1 for train fold, 0 for validation fold
    test_fold = np.concatenate(
        [
            -1 * np.ones(len(X_train), dtype=int),
            np.zeros(len(X_val), dtype=int),
        ]
    )
    ps = PredefinedSplit(test_fold=test_fold)

    # --- Hyperparameter search space (as agreed) ---
    param_distributions: dict[str, Any] = {
        "learning_rate": np.linspace(0.01, 0.2, 50),
        "max_depth": np.arange(3, 11),
        "subsample": np.linspace(0.6, 1.0, 50),
        "colsample_bytree": np.linspace(0.6, 1.0, 50),
    }

    model = XGBRegressor(
        objective=objective,
        n_estimators=int(base_params.get("n_estimators", 800)),
        tree_method=str(base_params.get("tree_method", "hist")),
        n_jobs=int(base_params.get("n_jobs", n_jobs)),
        random_state=int(base_params.get("random_state", random_state)),
        **{k: v for k, v in base_params.items() if k not in {"n_estimators", "tree_method", "n_jobs", "random_state"}},
    )

    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=int(n_iter),
        scoring="neg_mean_absolute_error",
        cv=ps,
        random_state=int(random_state),
        n_jobs=int(n_jobs),
        verbose=1,
        refit=True,  # refit on train+val with best params
    )

    # Fit search with weights (train weighted, val unweighted)
    search.fit(X_trv, y_trv, sample_weight=w_trv)

    best_params: dict[str, Any] = dict(search.best_params_)
    best_val_mae = float(-search.best_score_)

    print(f"Tuning objective: {objective}")
    print("Best params:", best_params)
    print(f"Best holdout (Val 2020) MAE: {best_val_mae:.4f}")

    # --- Train final model on TRAIN only (protocol guardrail) ---
    final_model = XGBRegressor(
        objective=objective,
        n_estimators=int(base_params.get("n_estimators", 800)),
        tree_method=str(base_params.get("tree_method", "hist")),
        n_jobs=int(base_params.get("n_jobs", n_jobs)),
        random_state=int(base_params.get("random_state", random_state)),
        **{k: v for k, v in base_params.items() if k not in {"n_estimators", "tree_method", "n_jobs", "random_state"}},
        **best_params,
    )

    # IMPORTANT: evaluate using your existing harness (no extra kwargs)
    res = run_and_evaluate_model(
        model=final_model,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        X_test=X_test,
        y_test=y_test,
        sample_weight_train=w_train,
    )

    # Normalize to dict + attach metadata OUTSIDE the harness
    if hasattr(res, "to_dict"):
        best_model_eval = res.to_dict()
    elif isinstance(res, dict):
        best_model_eval = dict(res)
    else:
        # e.g., pd.Series or dataclass-like without to_dict
        best_model_eval = dict(res)

    best_model_eval["objective"] = objective
    best_model_eval["tag"] = f"xgb_tuned_{objective}"
    best_model_eval["best_val_mae_from_search"] = best_val_mae
    best_model_eval["best_params"] = best_params

    return TuningResult(
        best_params=best_params,
        best_val_mae=best_val_mae,
        best_model_eval=best_model_eval,
    )



@dataclass(frozen=True)
class TuningResult:
    best_params: dict[str, Any]
    best_val_mae: float
    best_model: XGBRegressor

def tune_xgb_with_predefinedsplit_holdout(
    *,
    objective: str,
    X_train,
    y_train,
    w_train,
    X_val,
    y_val,
    base_params=None,
    n_iter: int = 30,
    random_state: int = 42,
    n_jobs: int = -1,
):
    """
    Randomized hyperparameter search using a strict train/validation holdout.
    Optimized CPU version with:
        - float32 arrays and no-copy conversions
        - early stopping to reduce unnecessary tree growth
        - joblib parallelization of the hyperparameter loop
        - no changes to the scientific protocol

    Parameters
    ----------
    objective : str
        XGBoost objective function (e.g., "reg:absoluteerror").
    X_train, y_train : pd.DataFrame / pd.Series
        Training data (already preprocessed).
    w_train : np.ndarray
        Sample weights for training.
    X_val, y_val : pd.DataFrame / pd.Series
        Validation data for model selection.
    base_params : dict or None
        Additional fixed parameters for XGBRegressor.
    n_iter : int
        Number of random hyperparameter samples.
    random_state : int
        Random seed for reproducibility.
    n_jobs : int
        Number of parallel jobs for joblib.

    Returns
    -------
    TuningResult
        Contains best_params, best_val_mae, and best_model.
    """

    if base_params is None:
        base_params = {}

    # --- Convert once outside the loop (much faster) ---
    Xtr = X_train.to_numpy(dtype=np.float32, copy=False)
    Xva = X_val.to_numpy(dtype=np.float32, copy=False)
    ytr = y_train.to_numpy(dtype=np.float32, copy=False)
    yva = y_val.to_numpy(dtype=np.float32, copy=False)
    wtr = w_train.astype(np.float32, copy=False)

    # --- Optimized search space ---
    param_dist = {
        "learning_rate": np.linspace(0.02, 0.10, 10),
        "max_depth": np.arange(4, 11),
        "subsample": np.linspace(0.7, 1.0, 10),
        "colsample_bytree": np.linspace(0.7, 1.0, 10),
        "min_child_weight": [1, 5, 10],
        "gamma": [0.0, 0.5, 1.0],
        "reg_alpha": [0.0, 1e-3, 1e-2, 1e-1],
        "reg_lambda": [1.0, 5.0, 10.0],
    }

    sampler = list(ParameterSampler(param_dist, n_iter=n_iter, random_state=random_state))

    # --- Function executed in parallel for each hyperparameter set ---
    def evaluate_params(params):
        model = XGBRegressor(
            objective=objective,
            n_estimators=2000,            # large upper bound
            early_stopping_rounds=50,     # stops early
            eval_metric="mae",
            tree_method="hist",           # CPU-optimized
            random_state=random_state,
            **base_params,
            **params,
        )

        model.fit(
            Xtr, ytr,
            sample_weight=wtr,
            eval_set=[(Xva, yva)],
            verbose=False,
        )

        pred_val = model.predict(Xva)
        val_mae = float(mean_absolute_error(yva, pred_val))
        return val_mae, params, model

    # --- Parallel execution of the tuning loop ---
    results = Parallel(n_jobs=n_jobs)(
        delayed(evaluate_params)(p) for p in sampler
    )

    # --- Select best model ---
    best_val_mae, best_params, best_model = min(results, key=lambda x: x[0])

    return TuningResult(
        best_params=best_params,
        best_val_mae=best_val_mae,
        best_model=best_model,
    )

    


def evaluate_with_clip_0_100(y_true: np.ndarray, y_pred: np.ndarray) -> tuple[float, float, float, float]:
    y_pred_clip = np.clip(y_pred.astype(float), 0.0, 100.0)
    mae = float(mean_absolute_error(y_true.astype(float), y_pred_clip))
    return mae, float(np.min(y_pred_clip)), float(np.max(y_pred_clip)), float(np.mean(y_pred_clip))



def _to_float_np(x):
    return x.to_numpy(dtype=float) if hasattr(x, "to_numpy") else np.asarray(x, dtype=float)



def eval_with_clip_0_100(y_true, y_pred):
    y_pred_c = np.clip(y_pred, 0.0, 100.0)
    return float(mean_absolute_error(y_true, y_pred_c))


def mae_clip(y_true, y_pred):
    return mean_absolute_error(y_true, np.clip(y_pred, 0.0, 100.0))



def _is_clip_tag(tag: str) -> bool:
    return "clip" in str(tag).lower()

def _sha256_file(path: Path) -> str:
    return hashlib.sha256(path.read_bytes()).hexdigest()





def evaluate_mode(mode_name, df):
    mode = MODES[mode_name]

    # Filter rows by tag semantics
    df_mode = df[mode["filter"](df)].copy()

    # Keep only rows with both Val and Test
    df_sel = df_mode.dropna(subset=[SEL_COL, TEST_COL]).copy()

    # Baseline
    baseline_tag = mode["baseline"]
    baseline_row = df_sel.loc[df_sel["tag"].astype(str).eq(baseline_tag)]
    if baseline_row.empty:
        raise KeyError(f"Baseline '{baseline_tag}' not found in mode '{mode_name}'.")
    baseline_row = baseline_row.iloc[0]
    baseline_test = float(baseline_row[TEST_COL])

    # Candidates (exclude baseline)
    df_cand = df_sel.loc[df_sel["tag"] != baseline_tag]

    # Best by validation
    best_row = df_cand.sort_values(SEL_COL, ascending=True).iloc[0]
    best_tag  = str(best_row["tag"])
    best_test = float(best_row[TEST_COL])

    # Champion
    champion_tag = best_tag if best_test < baseline_test else baseline_tag

    return {
        "mode": mode_name,
        "baseline_tag": baseline_tag,
        "baseline_test": baseline_test,
        "best_tag": best_tag,
        "best_test": best_test,
        "gap": best_test - baseline_test,
        "champion": champion_tag,
    }


## 1.6 - Load frozen artifacts (governance config + baseline audit + optional pack)

Cycle 3 has **two complementary “sources of truth”**:

1) **Cycle governance (what we decided and why)**  
   `frozen_config_cycle2.json` captures the Cycle 2 decisions that govern Cycle 3:
   - decision split definition (temporal),
   - reporting metrics,
   - the strategic pivot direction for Cycle 3.

2) **Baseline contract (what must be reproduced exactly)**  
   `baseline_huber15_audit_v3_from_pack.json` is the operational contract for the official baseline track:
   **Baseline_Huber15_recency0p05_medfill**.  
   It specifies:
   - the exact list of **15 numeric columns**,
   - Huber parameters,
   - recency-weighting settings,
   - and fingerprints/hashes for traceability.

3) **Optional baseline pack (final reproducibility anchor)**  
   The `.npz` pack is the *final* reproducibility anchor for the baseline (arrays + indices + weights).
   We **do not** use it as the main training path in Cycle 3 (to keep the pipeline “alive”),
   but we will use it later as an **audit gate** to confirm we can reproduce the baseline when needed.

Next, we load the governance config and the baseline audit v3, print a structured summary, and
validate that the baseline audit agrees with the Cycle 3 frozen protocol (columns / params / recency settings).


In [6]:
# --- Load governance config + baseline audit v3 ---
governance_cfg = load_json_dict(CYCLE2_MODELS_DIR / "frozen_config_cycle2.json")
baseline_audit_v3 = load_json_dict(FROZEN_CONFIG_PATH)

# --- Summaries ---
summarize_governance_config(governance_cfg)
summarize_baseline_audit_v3(baseline_audit_v3)

# --- Lightweight consistency checks ---
validate_cycle3_alignment(governance_cfg=governance_cfg, baseline_audit=baseline_audit_v3)

Loaded: frozen_config_cycle2.json
SHA256:  8aaebe8581946b1fe8268f4b8ee5fecb8e6307487977a12b324b164d642f0045
Loaded: baseline_huber15_audit_v3_from_pack.json
SHA256:  3915be4f9022e220d60ed24c4965906908b682eec3c130495858dc9891f50d12

[Governance config]
Top-level keys: ['baseline_protocols', 'cycle', 'cycle3_next_experiment', 'decision_split', 'description', 'feature_engineering_artifact_cycle2', 'frozen_track_cycle3', 'guardrail_split', 'hurdle_status_cycle2', 'metrics']
Cycle: 2
Description: Cycle 2 frozen config for Cycle 3 modeling (single-track: Huber-15 numeric-only). Engineered features kept as documented artifact but not the Cycle 3 benchmark track.

Decision split:
  - type: temporal
  - train: <=2019
  - val: 2020
  - test: 2021

Guardrail split:
  - type: random
  - train: 0.7
  - val: 0.1
  - test: 0.2

Metrics (reporting):
  - ['mae', 'rmse', 'r2', 'mae_zero', 'mae_pos', 'pct_zero_true', 'pred_zero_pct']

[Baseline audit v3]
Top-level keys: ['baseline_protocol', 'dtypes', 'h

# 2. Load Processed Dataset (Parquet)

In [7]:
# --- Load processed dataset (Parquet) ---
if not DATA_PROCESSED_PATH.exists():
    raise FileNotFoundError(
        "Processed dataset not found.\n"
        f"Expected at: {DATA_PROCESSED_PATH}\n"
        "Run the preprocessing pipeline to generate it, then re-run this notebook."
    )

df = pd.read_parquet(DATA_PROCESSED_PATH)

display(df.sample(5, random_state=RANDOM_SEED))
print("Shape:", df.shape)

# --- Minimal schema sanity checks (fail fast) ---
required_cols = ["album_release_year"]
missing_cols = [c for c in required_cols if c not in df.columns]
if missing_cols:
    raise KeyError(f"Missing required columns in processed dataset: {missing_cols}")

,song_popularity,album_release_year,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,song_explicit,speechiness,tempo,time_signature,total_available_markets,valence,release_year_missing_or_suspect
49516,48,2019,0.0466,0.4700,"213,159.0000",0.9610,0.0000,7,0.3050,-0.8680,1,False,0.3630,164.8990,4,168,0.6480,False
38956,51,2020,0.1130,0.7640,"344,201.0000",0.5670,0.0241,6,0.1050,-9.3880,0,False,0.0299,134.0250,4,170,0.6880,False
221529,20,1988,0.5350,0.3440,"321,627.0000",0.4590,0.0000,1,0.0677,-8.1630,1,False,0.0342,125.8110,4,170,0.2510,False
199218,22,2018,0.8730,0.6270,"133,933.0000",0.4680,0.0000,0,0.1690,-14.6700,1,False,0.9370,78.5860,4,170,0.2780,False
143101,30,2005,0.9330,0.1570,"741,160.0000",0.2240,0.9100,7,0.0748,-15.1410,1,False,0.0349,98.2000,4,170,0.0383,False


Shape: (439865, 18)


# 3. Baseline Check — Huber-15 (Official Cycle 3 Track)

Cycle 2 froze a **single official modeling track** for Cycle 3 to avoid mixing input spaces and to keep the benchmark fully reproducible:

**Protocol:** `Baseline_Huber15_recency0p05_medfill`  
- Input space: **15 raw numeric columns** (fixed list from the frozen config)  
- Preprocessing: **median imputation**, fit on **train only**  
- Temporal split: train `<=2019`, val `2020`, test `2021`  
- Recency weighting: enabled, `lambda=0.05`, `current_year=2021`  
- Baseline model: `HuberRegressor` (params frozen)

In this section, we will reconstruct the exact **train/val/test matrices** under the frozen protocol and confirm the baseline performance before moving to robust XGBoost.

## 3.1 - Temporal split masks (decision split) and protocol inputs

We enforce the frozen **temporal split** using `album_release_year` and select the **exact 15 numeric columns** defined by the Cycle 3 baseline protocol.

To keep the split deterministic and avoid leaking unknown years into the future, any missing/invalid `album_release_year` values are assigned to the **train** split (they are handled later by train-only median imputation).

At this stage we only prepare the raw matrices; imputation and recency weights will be applied next.


In [8]:
YEAR_COL = "album_release_year"

# Split parameters (frozen)
train_max_year = int(governance_cfg["decision_split"]["train"].replace("<=", ""))
val_year = int(governance_cfg["decision_split"]["val"])
test_year = int(governance_cfg["decision_split"]["test"])

train_mask, val_mask, test_mask = build_temporal_split_masks(
    df,
    year_col=YEAR_COL,
    train_max_year=train_max_year,
    val_year=val_year,
    test_year=test_year,
    nan_policy="train",
)

protocol_key = _extract_cycle3_protocol_key(governance_cfg)
numeric_cols = extract_protocol_columns(governance_cfg, protocol_key=protocol_key)

missing_cols = [c for c in numeric_cols if c not in df.columns]
if missing_cols:
    raise KeyError(f"Missing protocol columns in df: {missing_cols}")

TARGET_COL = "song_popularity"
if TARGET_COL not in df.columns:
    raise KeyError(f"Target column not found: {TARGET_COL}")

# Raw split matrices (no imputation yet)
X_train_raw = df.loc[train_mask, numeric_cols].copy()
X_val_raw = df.loc[val_mask, numeric_cols].copy()
X_test_raw = df.loc[test_mask, numeric_cols].copy()

y_train = df.loc[train_mask, TARGET_COL].astype(float).copy()
y_val = df.loc[val_mask, TARGET_COL].astype(float).copy()
y_test = df.loc[test_mask, TARGET_COL].astype(float).copy()

print("Protocol:", protocol_key)
print("Numeric cols:", len(numeric_cols))
print(
    "Split sizes:",
    {"train": int(train_mask.sum()), "val": int(val_mask.sum()), "test": int(test_mask.sum())},
)
print("X shapes:", X_train_raw.shape, X_val_raw.shape, X_test_raw.shape)
print("y shapes:", y_train.shape, y_val.shape, y_test.shape)
print("NaN years assigned to train:", int(pd.to_numeric(df[YEAR_COL], errors="coerce").isna().sum()))


Protocol: Baseline_Huber15_recency0p05_medfill
Numeric cols: 15
Split sizes: {'train': 283488, 'val': 105605, 'test': 50772}
X shapes: (283488, 15) (105605, 15) (50772, 15)
y shapes: (283488,) (105605,) (50772,)
NaN years assigned to train: 22


## 3.2 - Median imputation (fit on train only)

The protocol requires a `SimpleImputer(strategy="median")` fit **only on the training split**.
We apply the fitted imputer to train/val/test to produce the final numeric matrices used for modeling.


In [9]:
imputer = fit_train_only_median_imputer(X_train_raw)

X_train = transform_with_imputer(imputer, X_train_raw, columns=numeric_cols, index=X_train_raw.index)
X_val = transform_with_imputer(imputer, X_val_raw, columns=numeric_cols, index=X_val_raw.index)
X_test = transform_with_imputer(imputer, X_test_raw, columns=numeric_cols, index=X_test_raw.index)

print("Imputation applied (train-only fit).")
print("Any NaNs after imputation:", {"train": bool(X_train.isna().any().any()),
                                   "val": bool(X_val.isna().any().any()),
                                   "test": bool(X_test.isna().any().any())})


Imputation applied (train-only fit).
Any NaNs after imputation: {'train': False, 'val': False, 'test': False}


## 3.3 - Next: recency weighting + Huber fit (frozen params)

Cycle 2 selected a conservative recency weighting scheme (`lambda=0.05`) to account for concept drift without degrading 2021 performance.

We compute `sample_weight` **only for the training split**, using the frozen definition:

- `age = clip(current_year - album_release_year, lower=0)`
- `weight = exp(-lambda * age)`

These weights will be passed to `HuberRegressor.fit(..., sample_weight=weights)` as part of the official protocol.

In [10]:
lambda_recency = float(
    governance_cfg["baseline_protocols"][protocol_key]["recency_weighting"]["lambda"]
)
current_year = int(
    governance_cfg["baseline_protocols"][protocol_key]["recency_weighting"]["current_year"]
)

train_years = df.loc[train_mask, YEAR_COL]
sample_weight_train = compute_recency_weights(
    train_years,
    current_year=current_year,
    lambda_recency=lambda_recency,
)

print("Recency weights computed (train only).")
print("  X_train rows:", X_train.shape[0])
print("  w_train len :", sample_weight_train.shape[0])
if sample_weight_train.shape[0] != X_train.shape[0]:
    raise ValueError("sample_weight_train length does not match X_train rows.")

summarize_array(sample_weight_train, name="sample_weight_train")

Recency weights computed (train only).
  X_train rows: 283488
  w_train len : 283488
sample_weight_train:
  n=283488
  min=0.003028  p01=0.090718  p05=0.246597  p50=0.778801  p95=0.904837  p99=0.904837  max=1.000000
  mean=0.695242  std=0.212252


## 3.4 - Fit the frozen Huber baseline and evaluate (Val 2020 / Test 2021)

With:
- the frozen 15-column input space,
- train-only median imputation,
- and recency weights for training,

we can now fit `HuberRegressor` using the **frozen hyperparameters** from the config and evaluate:

- **Val (2020)**: to validate the decision split behavior.
- **Test (2021)**: the benchmark we must reproduce and later beat with robust XGBoost objectives.

We also report segmented MAE for `y==0` vs `y>0`, since 2021 has higher zero inflation.

This step is the baseline anchor for the rest of Cycle 3: every challenger will use the same split and input space.

In [201]:
huber_params = governance_cfg["baseline_protocols"][protocol_key]["model"]["params"]
huber = HuberRegressor(**huber_params)

huber.fit(
    X_train.to_numpy(dtype=float),
    y_train.to_numpy(dtype=float),
    sample_weight=sample_weight_train,
)

y_pred_val = huber.predict(X_val.to_numpy(dtype=float))
y_pred_test = huber.predict(X_test.to_numpy(dtype=float))

mae_val = evaluate_mae(y_val.to_numpy(dtype=float), y_pred_val)
mae_test = evaluate_mae(y_test.to_numpy(dtype=float), y_pred_test)

print("Huber baseline — Val 2020 MAE:", f"{mae_val:.4f}")
print("Huber baseline — Test 2021 MAE:", f"{mae_test:.4f}")
print(
    "Pred range (Test):",
    f"[{float(np.min(y_pred_test)):.4f}, {float(np.max(y_pred_test)):.4f}]",
)


Huber baseline — Val 2020 MAE: 15.2613
Huber baseline — Test 2021 MAE: 15.2127
Pred range (Test): [-15.6361, 31.3571]


# 4. The Challenger — Robust XGBoost (same frozen protocol as the baseline) 

With the baseline fully reconciled under the frozen Cycle 3 protocol (**Baseline_Huber15_recency0p05_medfill**: 15 numeric columns + train‑only median imputation + train‑only recency weights), we now execute the strategic pivot of Cycle 3: 

> Combine the **non‑linearity** of gradient‑boosted trees with **robust loss functions**
> to reduce sensitivity to outliers and improve generalization under the 2021 regime shift.

This section is intentionally structured as: 
1. **protocol guardrails** (what must remain invariant),
2. a **single evaluation harness** (to prevent accidental drift),
3. **point‑runs** (fast signal),
4. a **short interpretation** that determines the tuning direction.

---

## 4.1 - Common Training/Evaluation Harness + Protocol Guardrails

Before training any XGBoost model, we define a small, reusable harness that:

* takes a model instance,
* fits it on `(X_train, y_train)` with `sample_weight_train`,
* evaluates MAE on **Val 2020** and **Test 2021**,
* and returns a compact result dict for logging.

This avoids repeated code and prevents accidental deviations (e.g., forgetting weights, changing matrices, or mixing input spaces).

To ensure a fair comparison, **every XGBoost candidate** evaluated through this harness must reuse *exactly* the same frozen components as the baseline:

* **Decision split:**  
  train ≤ 2019, val = 2020, test = 2021  
  (with invalid/missing years assigned to **train**, as defined in the frozen protocol)

* **Input space:**  
  the frozen list of **15 raw numeric columns**

* **Preprocessing:**  
  `SimpleImputer(strategy="median")` fit on **train only**, applied to val/test

* **Training weights:**  
  recency weights (λ = 0.05) applied **only** on train

If any of these components change, we are no longer comparing models — **we are comparing protocols**.

In [36]:
protocol_key = _extract_cycle3_protocol_key(governance_cfg)
frozen_huber_params = governance_cfg["baseline_protocols"][protocol_key]["model"]["params"]

huber = HuberRegressor(**frozen_huber_params)

res_huber = run_and_evaluate_model(
    model=huber,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    sample_weight_train=sample_weight_train,
)

print(json.dumps(res_huber.to_dict(), indent=4))

{
    "model_name": "HuberRegressor",
    "objective": null,
    "used_sample_weight": true,
    "mae_val_2020": 15.261278957800723,
    "mae_test_2021": 15.212667070481631,
    "pred_min_test": -15.636086723517685,
    "pred_max_test": 31.357097696169706,
    "params": {
        "alpha": 0.0001,
        "epsilon": 1.35,
        "fit_intercept": true,
        "max_iter": 100,
        "tol": 1e-05,
        "warm_start": false
    }
}


## 4.2 - Point-runs (fast signal): three objectives

We start with three minimal XGBoost runs (“point-runs”) to obtain an initial signal under the frozen protocol.
At this stage we are not trying to win — we are trying to understand how each loss behaves in this dataset.

### 4.2.1 Experiment 1 (control / naive): `objective="reg:squarederror"`

We begin with the default squared-error objective as a control condition.  
Because squared error is sensitive to outliers and heavy-tailed residuals, we expect it to underperform in this setting.  
Still, it provides a useful baseline against which more robust objectives can be compared.

In [190]:
xgb_sq = XGBRegressor(objective="reg:squarederror", random_state=RANDOM_SEED, n_estimators=500) 
res_sq = run_and_evaluate_model( 
    model=xgb_sq,
    X_train=X_train, 
    y_train=y_train, 
    X_val=X_val, 
    y_val=y_val, 
    X_test=X_test, 
    y_test=y_test, 
    sample_weight_train=sample_weight_train, 
)
res_sq = res_sq.to_dict()
flat = {
    **{k: v for k, v in res_sq.items() if k != "params"},
    **{k: v for k, v in res_sq["params"].items() if pd.notna(v)}
}
pd.DataFrame([flat])

,model_name,objective,used_sample_weight,mae_val_2020,mae_test_2021,pred_min_test,pred_max_test,enable_categorical,n_estimators,random_state
0,XGBRegressor,reg:squarederror,True,14.4042,15.5807,-14.4951,64.6988,False,500,42


### 4.2.2 Experiment 2 (robust): `objective="reg:absoluteerror"`

We then switch to MAE-based boosting, which is inherently more robust to outliers and heavy-tailed residuals.  
Given this robustness, we expect improved performance relative to the squared-error control, particularly on the Test 2021 MAE.

In [219]:
xgb_abs = XGBRegressor(objective="reg:absoluteerror", random_state=RANDOM_SEED, n_estimators=500) 
res_abs = run_and_evaluate_model( 
    model=xgb_abs,
    X_train=X_train, 
    y_train=y_train, 
    X_val=X_val, 
    y_val=y_val, 
    X_test=X_test, 
    y_test=y_test, 
    sample_weight_train=sample_weight_train, 
)
res_abs = res_abs.to_dict()
flat = {
    **{k: v for k, v in res_abs.items() if k != "params"},
    **{k: v for k, v in res_abs["params"].items() if pd.notna(v)}
}
pd.DataFrame([flat])

,model_name,objective,used_sample_weight,mae_val_2020,mae_test_2021,pred_min_test,pred_max_test,enable_categorical,n_estimators,random_state
0,XGBRegressor,reg:absoluteerror,True,14.2693,15.7063,-13.5263,63.2540,False,500,42


### 4.2.3 - Experiment 3 (robust): `objective="reg:pseudohubererror"`

Finally, we test the pseudo-Huber objective, which behaves like L2 near zero and transitions toward L1 for large residuals.  
This hybrid structure offers a compromise between stability and robustness, potentially bridging the performance gap between squared-error and absolute-error losses.

In [89]:
xgb_ph = XGBRegressor(objective="reg:pseudohubererror", random_state=RANDOM_SEED, n_estimators=500) 
res_ph = run_and_evaluate_model( 
    model=xgb_ph,
    X_train=X_train, 
    y_train=y_train, 
    X_val=X_val, 
    y_val=y_val, 
    X_test=X_test, 
    y_test=y_test, 
    sample_weight_train=sample_weight_train, 
)
res_ph = res_ph.to_dict()
flat = {
    **{k: v for k, v in res_ph.items() if k != "params"},
    **{k: v for k, v in res_ph["params"].items() if pd.notna(v)}
}
pd.DataFrame([flat])

,model_name,objective,used_sample_weight,mae_val_2020,mae_test_2021,pred_min_test,pred_max_test,enable_categorical,n_estimators,random_state
0,XGBRegressor,reg:pseudohubererror,True,15.0095,15.8873,-295.9467,260.7536,False,500,42


---

## 4.3 - Interpretation of point-runs

We compare these results to the frozen baseline:

* **Baseline (Huber-15 protocol):** Test 2021 MAE = **15.2127**

Key takeaways:

1. **None of the point-runs surpasses the baseline on Test 2021.**  
   Although all objectives achieve stronger Val 2020 MAE than the baseline, this advantage does not translate to 2021.  
   The squared-error run (15.58), absolute-error run (15.71), and pseudo-Huber run (15.89) all underperform relative to 15.21.

2. **Val improves while Test degrades**, reinforcing the presence of **temporal/regime generalization difficulty**.  
   This aligns with the known 2021 distribution shift (increased zero-inflation and heavier tails), which penalizes objectives that overfit to 2020 structure.

3. **Pseudo-Huber is unstable under default settings.**  
   Its prediction range is extreme (≈ −296 to +261), far beyond the other objectives (≈ −14 to +65).  
   This indicates insufficient regularization or an unsuitable parameterization for this dataset; it should not be used without additional constraints.

Operational conclusion:  
We proceed to a controlled tuning phase focused on **generalization under the frozen protocol**, using MAE-driven selection and applying stronger regularization to stabilize robust objectives.

---

## 4.4 - Tuning with `RandomizedSearchCV` under a Pure Holdout Protocol (MAE)

The point-runs established that default XGBoost configurations do not surpass the frozen Huber-15 baseline on **Test 2021**.  
To investigate whether improved regularization can reduce the generalization gap, we now perform a controlled hyperparameter search.

Crucially, we adopt **Pure holdout**:  
**Train (≤2019)** is used for fitting all candidates, **Val 2020** is used exclusively for model selection, and **Test 2021** remains a strictly untouched holdout for final evaluation.  
This preserves the temporal structure of the problem and prevents any feedback loop with the test year.

### Guardrails (unchanged)

* **Temporal split:** train ≤ 2019, val = 2020, test = 2021  
* **Input space:** 15 numeric columns (Huber-15 protocol)  
* **Imputation:** median, fit on train-only  
* **Weights:** recency weights (λ = 0.05), applied to train-only  
* **Selection metric:** `neg_mean_absolute_error` (i.e., minimize MAE on Val 2020)

### Tuning strategy (and rationale)

We tune only the hyperparameters that directly control model capacity and regularization:

* `learning_rate`  
* `max_depth`  
* `subsample`  
* `colsample_bytree`

To maintain the integrity of the temporal split, we avoid internal cross-validation folds.  
Instead, we construct a **single predefined split** (train vs. val) and run:

* `RandomizedSearchCV(n_iter=20, scoring="neg_mean_absolute_error", refit=False)`  
* with a **PredefinedSplit** ensuring that Val 2020 is the sole selection reference.

This design prevents year mixing, avoids diluting the drift signal, and keeps Test 2021 fully isolated.

### Final model 

After identifying the best hyperparameters, we **refit the final model on Train (≤2019) only**, not on train+val.  
Val 2020 may be reported as a post-hoc sanity check, but it does not participate in training the final estimator.

### Expected outputs

* Best hyperparameter configuration (`best_params_`)  
* Val 2020 MAE of the selected configuration (`best_score_`)  
* Final refit on Train (≤2019)  
* Evaluation on Test 2021 (MAE + prediction range)  
* Comparison against the baseline: success if `MAE_test_2021 < 15.2127`

In [103]:
# 1) Tuning - squarederror
res_tuned_sq = tune_xgb_with_predefinedsplit_holdout(
    objective="reg:squarederror",
    X_train=X_train,
    y_train=y_train,
    w_train=sample_weight_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    n_iter=20,
    random_state=RANDOM_SEED,
)
flat = {
    **{k: v for k, v in res_tuned_sq.best_model_eval.items()
       if k not in ("best_params", "params")},
    **{k: v for k, v in res_tuned_sq.best_model_eval["params"].items()
       if pd.notna(v)}
}
pd.DataFrame([flat]).T

Fitting 1 folds for each of 20 candidates, totalling 20 fits
Tuning objective: reg:squarederror
Best params: {'subsample': np.float64(0.6653061224489796), 'max_depth': np.int64(10), 'learning_rate': np.float64(0.025510204081632654), 'colsample_bytree': np.float64(0.6489795918367347)}
Best holdout (Val 2020) MAE: 14.1789


,0
model_name,XGBRegressor
objective,reg:squarederror
used_sample_weight,True
mae_val_2020,14.1789
mae_test_2021,15.4564
pred_min_test,-3.2927
pred_max_test,54.7270
tag,xgb_tuned_reg:squarederror
best_val_mae_from_search,14.1789
colsample_bytree,0.6490


In [104]:
# 2) Tuning - absoluteerror
res_tuned_abs = tune_xgb_with_predefinedsplit_holdout(
    objective="reg:absoluteerror",
    X_train=X_train,
    y_train=y_train,
    w_train=sample_weight_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
)
flat = {
    **{k: v for k, v in res_tuned_abs.best_model_eval.items()
       if k not in ("best_params", "params")},
    **{k: v for k, v in res_tuned_abs.best_model_eval["params"].items()
       if pd.notna(v)}
}
pd.DataFrame([flat]).T

Fitting 1 folds for each of 20 candidates, totalling 20 fits
Tuning objective: reg:absoluteerror
Best params: {'subsample': np.float64(0.6653061224489796), 'max_depth': np.int64(10), 'learning_rate': np.float64(0.025510204081632654), 'colsample_bytree': np.float64(0.6489795918367347)}
Best holdout (Val 2020) MAE: 14.1279


,0
model_name,XGBRegressor
objective,reg:absoluteerror
used_sample_weight,True
mae_val_2020,14.1279
mae_test_2021,15.5150
pred_min_test,-3.2848
pred_max_test,55.2573
tag,xgb_tuned_reg:absoluteerror
best_val_mae_from_search,14.1279
colsample_bytree,0.6490


In [115]:
# 3) Tuning - pseudohubererror
res_tuned_ph  = tune_xgb_with_predefinedsplit_holdout(
    objective="reg:pseudohubererror",
    X_train=X_train,
    y_train=y_train,
    w_train=sample_weight_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
)
flat = {
    **{k: v for k, v in res_tuned_ph.best_model_eval.items()
       if k not in ("best_params", "params")},
    **{k: v for k, v in res_tuned_ph.best_model_eval["params"].items()
       if pd.notna(v)}
}
pd.DataFrame([flat]).T

Fitting 1 folds for each of 20 candidates, totalling 20 fits
Tuning objective: reg:pseudohubererror
Best params: {'subsample': np.float64(0.6653061224489796), 'max_depth': np.int64(10), 'learning_rate': np.float64(0.025510204081632654), 'colsample_bytree': np.float64(0.6489795918367347)}
Best holdout (Val 2020) MAE: 14.1467


,0
model_name,XGBRegressor
objective,reg:pseudohubererror
used_sample_weight,True
mae_val_2020,14.1467
mae_test_2021,15.3044
pred_min_test,-3.2895
pred_max_test,58.3352
tag,xgb_tuned_reg:pseudohubererror
best_val_mae_from_search,14.1467
colsample_bytree,0.6490


## 4.5 — Final comparison (Val 2020 vs Test 2021) and Cycle 3 decision

We now consolidate all candidates trained under the exact same frozen protocol
(**Huber-15 numeric-only + train-only median imputation + train-only recency weights**).

**Selection rule (methodological):**
- **Val 2020** is used for tuning/selection (no peeking into Test).
- **Test 2021** is the final holdout used only once to decide whether the challenger truly beats the baseline.

**Cycle 3 success criterion:**
The best challenger must achieve **MAE(Test 2021) < 15.2127** (Huber-15 baseline).


In [277]:
rows = [
    dict(res_huber.to_dict(), tag="baseline_huber15"),
    dict(res_sq,  tag="xgb_point_reg:squarederror"),
    dict(res_abs, tag="xgb_point_reg:absoluteerror"),
    dict(res_ph,  tag="xgb_point_reg:pseudohubererror"),
    dict(res_tuned_sq.best_model_eval,  tag="xgb_tuned_reg:squarederror"),
    dict(res_tuned_abs.best_model_eval, tag="xgb_tuned_reg:absoluteerror"),
    dict(res_tuned_ph.best_model_eval,  tag="xgb_tuned_reg:pseudohubererror"),
]

df_results = pd.DataFrame(rows)

cols_order = [
    "model_name", "objective", "tag", "used_sample_weight",
    "mae_val_2020", "mae_test_2021",
    "pred_min_test", "pred_max_test",
]
df_results = df_results[[c for c in cols_order if c in df_results.columns]]
df_results = df_results.sort_values("mae_test_2021").reset_index(drop=True)

display(df_results)

# --- Baseline / candidates split (anchor by tag) ---
df_base = df_results[df_results["tag"].eq("baseline_huber15")]
if df_base.shape[0] != 1:
    raise ValueError(f"Expected exactly 1 baseline row. Found {df_base.shape[0]}.")

baseline_mae = float(df_base["mae_test_2021"].iloc[0])

df_cand = df_results[~df_results["tag"].eq("baseline_huber15")].copy()
df_cand["gap_to_baseline"] = df_cand["mae_test_2021"] - baseline_mae

best_cand_row = df_cand.loc[df_cand["mae_test_2021"].idxmin()]
best_cand_mae = float(best_cand_row["mae_test_2021"])

print(f"Baseline (Huber-15) MAE Test 2021: {baseline_mae:.4f}")
print(f"Best candidate MAE Test 2021:      {best_cand_mae:.4f}")

if best_cand_mae < baseline_mae:
    print("Success criterion met: True (baseline beaten)")
else:
    closest_row = df_cand.iloc[df_cand["gap_to_baseline"].abs().argmin()]
    print("Success criterion met: False (baseline NOT beaten)")
    print(
        f"Closest candidate was: {closest_row['tag']} "
        f"with MAE={closest_row['mae_test_2021']:.4f} "
        f"(gap={closest_row['gap_to_baseline']:.4f})"
    )

,model_name,objective,tag,used_sample_weight,mae_val_2020,mae_test_2021,pred_min_test,pred_max_test
0,HuberRegressor,None,baseline_huber15,True,15.2613,15.2127,-15.6361,31.3571
1,XGBRegressor,reg:pseudohubererror,xgb_tuned_reg:pseudohubererror,True,14.1467,15.3044,-3.2895,58.3352
2,XGBRegressor,reg:squarederror,xgb_tuned_reg:squarederror,True,14.1789,15.4564,-3.2927,54.7270
3,XGBRegressor,reg:absoluteerror,xgb_tuned_reg:absoluteerror,True,14.1279,15.5150,-3.2848,55.2573
4,XGBRegressor,reg:squarederror,xgb_point_reg:squarederror,True,14.4042,15.5807,-14.4951,64.6988
5,XGBRegressor,reg:absoluteerror,xgb_point_reg:absoluteerror,True,14.2693,15.7063,-13.5263,63.2540
6,XGBRegressor,reg:pseudohubererror,xgb_point_reg:pseudohubererror,True,15.0095,15.8873,-295.9467,260.7536


Baseline (Huber-15) MAE Test 2021: 15.2127
Best candidate MAE Test 2021:      15.3044
Success criterion met: False (baseline NOT beaten)
Closest candidate was: xgb_tuned_reg:pseudohubererror with MAE=15.3044 (gap=0.0917)


### Decision

Under the frozen Cycle 3 protocol, none of the XGBoost candidates (including tuned robust objectives)
improved upon the **Huber-15 baseline** on the **Test 2021** holdout.

Therefore, the Cycle 3 champion remains:

- **Baseline_Huber15_recency0p05_medfill** (MAE Test 2021 = 15.2127)

The XGBoost models are kept as documented experiments, but they are **not** promoted as the Cycle 3 best model.


## 4.6 - Final Robustness Sweep: Confirming the Baseline’s Dominance

This section documents the full experimental trajectory that led to the final model selection.  
Each step includes:

- the **code used**,  
- the **motivation behind the experiment**,  
- and the **result that justified moving to the next stage**.

This creates a transparent and reproducible narrative showing why the Huber‑15 baseline ultimately remained the strongest model under the 2020→2021 drift.

---

### **4.6.1 Establishing the Baseline (Huber‑15)**

We begin with a robust linear baseline using the Huber loss with parameter (ε = 15).

This model is intentionally simple, convex, and resistant to outliers — a natural anchor for evaluating drift robustness.

In [258]:
y_pred_test_clip = np.clip(y_pred_test, 0.0, 100.0)

mae_test_no_clip = mean_absolute_error(y_test.to_numpy(float), y_pred_test)
mae_test_clip = mean_absolute_error(y_test.to_numpy(float), y_pred_test_clip)

print(f"MAE Test 2021 (no clip): {mae_test_no_clip:.4f}")
print(f"MAE Test 2021 (clip 0-100): {mae_test_clip:.4f}")
print(f"Delta (clip - no clip): {mae_test_clip - mae_test_no_clip:+.4f}")

MAE Test 2021 (no clip): 15.2127
MAE Test 2021 (clip 0-100): 15.2000
Delta (clip - no clip): -0.0127


In [311]:
row_huber_clip = {
    "model_name": "HuberRegressor",
    "objective": None,
    "tag": "baseline_huber15_clip",
    "used_sample_weight": True,
    "mae_val_2020": res_huber.to_dict().get("mae_val_2020"),
    "mae_test_2021": mae_test_clip,
    "pred_min_test": float(y_pred_test_clip.min()),
    "pred_max_test": float(y_pred_test_clip.max()),
}
df_results = pd.concat([df_results, pd.DataFrame([row_huber_clip])], ignore_index=True)
df_results = df_results.drop_duplicates(subset=["tag"], keep="first")

This value becomes the **target to beat** for all subsequent models.

---

### **4.6.2 Exploring Alternative Model Families**

Before moving to boosted trees, we evaluated whether other model classes could naturally outperform the baseline under drift.

#### **1. Hurdle Model (Two‑Stage Zero Inflation)**  

In [262]:
thr = 0.5

# ---- Stage 1: classifier (train) ----
y_train_bin = (y_train.to_numpy(float) > 0).astype(int)
y_val_bin   = (y_val.to_numpy(float) > 0).astype(int)
y_test_bin  = (y_test.to_numpy(float) > 0).astype(int)

clf = LogisticRegression(max_iter=200, class_weight="balanced", n_jobs=-1)
clf.fit(X_train.to_numpy(float), y_train_bin, sample_weight=sample_weight_train)

p_test_pos = clf.predict_proba(X_test.to_numpy(float))[:, 1]

# ---- Stage 2: regressor on positives only (train) ----
pos_mask_train = y_train.to_numpy(float) > 0
reg = HuberRegressor(**huber_params)
reg.fit(
    X_train.loc[pos_mask_train].to_numpy(float),
    y_train.loc[pos_mask_train].to_numpy(float),
    sample_weight=sample_weight_train[pos_mask_train],
)

y_pred_test_reg = reg.predict(X_test.to_numpy(float))

# ---- Combine ----
y_pred_test_hurdle = np.where(p_test_pos >= thr, y_pred_test_reg, 0.0)
y_pred_test_hurdle = np.clip(y_pred_test_hurdle, 0.0, 100.0)

mae_test_hurdle = mean_absolute_error(y_test.to_numpy(float), y_pred_test_hurdle)

pred_zero_pct = float((y_pred_test_hurdle == 0.0).mean() * 100)
true_zero_pct = float((y_test.to_numpy(float) == 0.0).mean() * 100)

print(f"Hurdle(min) MAE Test 2021: {mae_test_hurdle:.4f}")
print(f"True zero%: {true_zero_pct:.2f} | Pred zero%: {pred_zero_pct:.2f} | thr={thr}")


/home/daniel/.cache/pypoetry/virtualenvs/popforecast-dRaaJELz-py3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Hurdle(min) MAE Test 2021: 17.6846
True zero%: 23.87 | Pred zero%: 34.33 | thr=0.5


In [312]:
# --- Criar linha para o hurdle ---
row_hurdle = {
    "model_name": "Hurdle(LogReg + Huber)",
    "objective": "hurdle",
    "tag": "hurdle_logreg_huber_clip",
    "used_sample_weight": True,
    "mae_val_2020": res_huber.to_dict().get("mae_val_2020"),
    "mae_test_2021": mae_test_hurdle,
    "pred_min_test": float(y_pred_test_hurdle.min()),
    "pred_max_test": float(y_pred_test_hurdle.max()),
}

# --- Adicionar ao df_results ---
df_results = pd.concat([df_results, pd.DataFrame([row_hurdle])], ignore_index=True)

# --- Remover duplicadas por tag ---
df_results = df_results.drop_duplicates(subset=["tag"], keep="first")

The model over‑predicts zeros and performs substantially worse than the baseline.

#### **2. Tweedie Regressors (GLM Family)**

In [142]:
# 1) Scale (fit on train only)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train.to_numpy(float))
X_val_s = scaler.transform(X_val.to_numpy(float))
X_test_s = scaler.transform(X_test.to_numpy(float))

# 2) Try a small sweep on power (keep it minimal)
powers = [1.0, 1.2, 1.5, 1.8]   # minimal, interpretable sweep
rows = []

for p in powers:
    model = TweedieRegressor(
        power=p,
        alpha=0.0,          # start simple; add regularization only if it converges
        link="log",
        max_iter=2000,
        tol=1e-6,
    )

    # TweedieRegressor supports sample_weight
    model.fit(X_train_s, y_train.to_numpy(float), sample_weight=sample_weight_train)

    pred_val = model.predict(X_val_s)
    pred_test = model.predict(X_test_s)

    # Clip to domain (0–100) to match your “defensible” reporting choice
    pred_val_c = np.clip(pred_val, 0.0, 100.0)
    pred_test_c = np.clip(pred_test, 0.0, 100.0)

    mae_val = float(mean_absolute_error(y_val.to_numpy(float), pred_val_c))
    mae_test = float(mean_absolute_error(y_test.to_numpy(float), pred_test_c))

    rows.append(
        {
            "model": "TweedieRegressor",
            "power": p,
            "mae_val_2020_clip": mae_val,
            "mae_test_2021_clip": mae_test,
            "pred_min_test": float(np.min(pred_test)),
            "pred_max_test": float(np.max(pred_test)),
        }
    )

df_tweedie = pd.DataFrame(rows).sort_values("mae_test_2021_clip").reset_index(drop=True)
display(df_tweedie)


,model,power,mae_val_2020_clip,mae_test_2021_clip,pred_min_test,pred_max_test
0,TweedieRegressor,1.0000,15.2135,15.5129,3.7809,41.5472
1,TweedieRegressor,1.2000,15.2108,15.5369,4.2808,41.7994
2,TweedieRegressor,1.5000,15.2073,15.5736,5.1983,42.2427
3,TweedieRegressor,1.8000,15.2044,15.6097,6.1842,42.7838


None approached the baseline.

In [313]:
rows_tweedie_for_df = []

for _, r in df_tweedie.iterrows():
    p = r["power"]
    rows_tweedie_for_df.append({
        "model_name": "TweedieRegressor",
        "objective": f"tweedie_power_{p}",
        "tag": f"tweedie_power_{p}",
        "used_sample_weight": True,
        "mae_val_2020": r["mae_val_2020_clip"],
        "mae_test_2021": r["mae_test_2021_clip"],
        "pred_min_test": r["pred_min_test"],
        "pred_max_test": r["pred_max_test"],
    })

df_results = pd.concat([df_results, pd.DataFrame(rows_tweedie_for_df)], ignore_index=True)

# garantir que não duplica
df_results = df_results.drop_duplicates(subset=["tag"], keep="first").reset_index(drop=True)

---

### **4.6.3 First Wave of XGBoost Experiments (Point‑Runs)**

In [282]:
models = {
    "xgb_point_squarederror": xgb_sq,
    "xgb_point_absoluteerror": xgb_abs,
    "xgb_point_pseudohuber": xgb_ph,
}

# reconstrói tuned models a partir de best_params
models["xgb_tuned_squarederror"] = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=800,
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
    **res_tuned_sq.best_params,
)
models["xgb_tuned_absoluteerror"] = XGBRegressor(
    objective="reg:absoluteerror",
    n_estimators=800,
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
    **res_tuned_abs.best_params,
)
models["xgb_tuned_pseudohuber"] = XGBRegressor(
    objective="reg:pseudohubererror",
    n_estimators=800,
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
    **res_tuned_ph.best_params,
)

# importante: fazer fit antes de predict
for tag, model in models.items():
    if tag.startswith("xgb_tuned_"):
        model.fit(
            X_train.to_numpy(float),
            y_train.to_numpy(float),
            sample_weight=sample_weight_train,
        )

rows = []
for tag, model in models.items():
    pred_val = model.predict(X_val.to_numpy(float))
    pred_test = model.predict(X_test.to_numpy(float))
    pred_val_c = np.clip(pred_val, 0.0, 100.0)
    pred_test_c = np.clip(pred_test, 0.0, 100.0)

    rows.append({
        "tag": tag,
        "objective": getattr(model, "objective", None),
        "mae_val_2020_clip": eval_with_clip_0_100(y_val.to_numpy(float), pred_val),
        "mae_test_2021_clip": eval_with_clip_0_100(y_test.to_numpy(float), pred_test),
        "pred_min_test": float(pred_test_c.min()),   # <-- CORRIGIDO
        "pred_max_test": float(pred_test_c.max()),   # <-- CORRIGIDO
    })

df_xgb_clip = pd.DataFrame(rows).sort_values("mae_test_2021_clip").reset_index(drop=True)
display(df_xgb_clip)


,tag,objective,mae_val_2020_clip,mae_test_2021_clip,pred_min_test,pred_max_test
0,xgb_tuned_pseudohuber,reg:pseudohubererror,14.1455,15.3036,0.0000,58.3352
1,xgb_tuned_squarederror,reg:squarederror,14.1788,15.4563,0.0000,54.7270
2,xgb_tuned_absoluteerror,reg:absoluteerror,14.1274,15.5146,0.0000,55.2573
3,xgb_point_squarederror,reg:squarederror,14.3968,15.5734,0.0000,64.6988
4,xgb_point_absoluteerror,reg:absoluteerror,14.2578,15.6953,0.0000,63.2540
5,xgb_point_pseudohuber,reg:pseudohubererror,14.6141,15.8018,0.0000,100.0000


In [314]:
df_xgb_clip_for_merge = df_xgb_clip.rename(columns={
    "mae_val_2020_clip": "mae_val_2020",
    "mae_test_2021_clip": "mae_test_2021",
})

df_xgb_clip_for_merge["model_name"] = "XGBRegressor"
df_xgb_clip_for_merge["used_sample_weight"] = True
df_xgb_clip_for_merge["tag"] = df_xgb_clip_for_merge["tag"] + "_clip"
cols = [
    "model_name",
    "objective",
    "tag",
    "used_sample_weight",
    "mae_val_2020",
    "mae_test_2021",
    "pred_min_test",
    "pred_max_test",
]

df_xgb_clip_for_merge = df_xgb_clip_for_merge[cols]
df_results = pd.concat([df_results, df_xgb_clip_for_merge], ignore_index=True)
df_results = df_results.sort_values("mae_test_2021").reset_index(drop=True)

We next evaluated XGBoost with fixed hyperparameters (“point‑runs”) to test whether non‑linear models could naturally outperform the baseline. These models were stable but clearly inferior to the baseline.

We then performed a RandomizedSearchCV over a moderate hyperparameter space. For the first time, a model approached the baseline — but still did not surpass it.

---

### **4.6.4 Expanded Hyperparameter Search (Aggressive Tuning)**

To ensure that the search space was not limiting performance, we expanded the tuning space substantially.

In [145]:
# --- Expanded XGB tuning space (still: PredefinedSplit holdout train/val; MAE; same protocol) ---

# --- How to use (repeat for the objectives you care about) ---
# Assumes you already have: X_train, y_train, X_val, y_val, X_test, y_test, sample_weight_train
# from the Baseline_Huber15_recency0p05_medfill protocol.

objectives_to_try = [
    "reg:squarederror",
    "reg:absoluteerror",
    "reg:pseudohubererror",
]

rows = []

for obj in objectives_to_try:
    res = tune_xgb_with_predefinedsplit_holdout(
        objective=obj,
        X_train=X_train,
        y_train=y_train,
        w_train=sample_weight_train,
        X_val=X_val,
        y_val=y_val,
        n_iter=30,          # bump to 30 since we expanded the space
        random_state=42,
        n_jobs=-1,
    )

    # Evaluate best model on val/test with clip 0-100 (same as your recent reporting)
    Xva = X_val.to_numpy(dtype=float)
    Xte = X_test.to_numpy(dtype=float)
    yva = y_val.to_numpy(dtype=float)
    yte = y_test.to_numpy(dtype=float)

    pred_val = res.best_model.predict(Xva)
    pred_test = res.best_model.predict(Xte)

    mae_val_clip, _, _, _ = evaluate_with_clip_0_100(yva, pred_val)
    mae_test_clip, pmin, pmax, _ = evaluate_with_clip_0_100(yte, pred_test)

    rows.append(
        {
            "tag": f"xgb_tuned_expanded_{obj}",
            "objective": obj,
            "best_val_mae_from_search": res.best_val_mae,   # raw (no clip) val MAE used for selection
            "mae_val_2020_clip": mae_val_clip,
            "mae_test_2021_clip": mae_test_clip,
            "pred_min_test_clip": pmin,
            "pred_max_test_clip": pmax,
            **res.best_params,
        }
    )

df_xgb_tuned_expanded = pd.DataFrame(rows).sort_values("mae_test_2021_clip").reset_index(drop=True)
display(df_xgb_tuned_expanded)

,tag,objective,best_val_mae_from_search,mae_val_2020_clip,mae_test_2021_clip,pred_min_test_clip,pred_max_test_clip,subsample,reg_lambda,reg_alpha,min_child_weight,max_depth,learning_rate,gamma,colsample_bytree
0,xgb_tuned_expanded_reg:absoluteerror,reg:absoluteerror,14.1302,14.1280,15.2541,0.0000,55.0586,0.8667,1.0000,0.1000,20,10,0.0450,1.0000,0.8833
1,xgb_tuned_expanded_reg:pseudohubererror,reg:pseudohubererror,14.1440,14.1431,15.3699,0.0000,54.9421,0.6833,5.0000,0.1000,2,8,0.0325,0.5000,0.7833
2,xgb_tuned_expanded_reg:squarederror,reg:squarederror,14.1689,14.1685,15.4683,0.0000,52.6271,0.6833,5.0000,0.1000,2,8,0.0325,0.5000,0.7833


**Representative result:**

```
Best params: {subsample=0.8667, reg_lambda=1.0, reg_alpha=0.1,
              min_child_weight=20, max_depth=10, learning_rate=0.045,
              gamma=1.0, colsample_bytree=0.8833}

Val 2020 MAE (clip): 14.1280
Test 2021 MAE (clip): 15.2541
```

This was the **closest any model came** to beating the baseline — only **0.04** above it.

But still: **not better**.

In [315]:
df_expanded = df_xgb_tuned_expanded.rename(columns={
    "mae_val_2020_clip": "mae_val_2020",
    "mae_test_2021_clip": "mae_test_2021",
    "pred_min_test_clip": "pred_min_test",
    "pred_max_test_clip": "pred_max_test",
})
df_expanded["model_name"] = "XGBRegressor"
df_expanded["used_sample_weight"] = True
df_expanded["tag"] = df_expanded["tag"].astype(str)
cols = [
    "model_name",
    "objective",
    "tag",
    "used_sample_weight",
    "mae_val_2020",
    "mae_test_2021",
    "pred_min_test",
    "pred_max_test",
]

df_expanded = df_expanded[cols]
df_results = pd.concat([df_results, df_expanded], ignore_index=True)
df_results = df_results.sort_values("mae_test_2021").reset_index(drop=True)

---

### **4.6.5 Additional Tuning Attempts (n_iter = 30)**

To ensure that the expanded search space had been sufficiently explored, we conducted a series of additional tuning attempts using **30 random samples** from a broader hyperparameter distribution. These experiments included:

- a full 30‑iteration search using the predefined split (val + test),  
- a separate 30‑iteration manual search evaluated **only on Val 2020**,  
- several refits using the best configurations found,  
- and targeted variations (simplified models, deeper models, and early‑stopping refits).

Across all attempts, **none of the models surpassed the Huber‑15 baseline**.


#### **1. Full 100‑iteration expanded search (val + test)**  
**Objective:** `reg:absoluteerror`  
**n_iter:** 30  

In [294]:
res_abs_expanded_100 = tune_xgb_with_predefinedsplit_holdout(
    objective="reg:absoluteerror",
    X_train=X_train, y_train=y_train,
    w_train=sample_weight_train,
    X_val=X_val, y_val=y_val,
    n_iter=30,
    random_state=42,
    n_jobs=-1,
)

pred_val = res_abs_expanded_100.best_model.predict(X_val.to_numpy(float))
pred_test = res_abs_expanded_100.best_model.predict(X_test.to_numpy(float))

mae_val_clip, _, _, _ = evaluate_with_clip_0_100(y_val.to_numpy(float), pred_val)
mae_test_clip, pmin, pmax, _ = evaluate_with_clip_0_100(y_test.to_numpy(float), pred_test)

print("Best params (abs, expanded, n_iter=100):", res_abs_expanded_100.best_params)
print("Val 2020 MAE (clip):", f"{mae_val_clip:.4f}")
print("Test 2021 MAE (clip):", f"{mae_test_clip:.4f}")
print("Pred range test (clip):", f"[{pmin:.4f}, {pmax:.4f}]")

Best params (abs, expanded, n_iter=100): {'subsample': np.float64(0.9), 'reg_lambda': 5.0, 'reg_alpha': 0.001, 'min_child_weight': 5, 'max_depth': np.int64(9), 'learning_rate': np.float64(0.028888888888888888), 'gamma': 1.0, 'colsample_bytree': np.float64(0.7)}
Val 2020 MAE (clip): 14.1227
Test 2021 MAE (clip): 15.3649
Pred range test (clip): [0.0000, 54.3557]


In [316]:
row_abs_expanded_30 = {
    "model_name": "XGBRegressor",
    "objective": "reg:absoluteerror",
    "tag": "xgb_tuned_expanded_abs_n30_clip",
    "used_sample_weight": True,
    "mae_val_2020": mae_val_clip,
    "mae_test_2021": mae_test_clip,
    "pred_min_test": float(pmin),
    "pred_max_test": float(pmax),
}
df_results = pd.concat([df_results, pd.DataFrame([row_abs_expanded_30])], ignore_index=True)
df_results = df_results.sort_values("mae_test_2021").reset_index(drop=True)

This configuration did **not** improve upon the best result from the earlier expanded search (15.2541), and remained above the baseline (15.2000).

#### **2. Manual 30‑iteration search (validation‑only)**  

To complement the predefined‑split search, we also ran a manual 30‑iteration exploration over a wider hyperparameter space, evaluating each configuration only on the 2020 validation set. This experiment was not intended to produce a final model, but rather to probe whether the expanded space contained regions with systematically lower validation error.

In [298]:
# Assumes you already have:
# - X_train, y_train, sample_weight_train
# - X_val, y_val
# - evaluate_with_clip_0_100(y_true_np, y_pred_np)

# Convert once (faster)
Xtr = _to_float_np(X_train)
ytr = _to_float_np(y_train)
wtr = _to_float_np(sample_weight_train)

Xva = _to_float_np(X_val)
yva = _to_float_np(y_val)

objective = "reg:absoluteerror"
random_state = 42
n_iter = 30   

base_params = dict(
    objective=objective,
    n_estimators=2000,          # large upper bound; early stopping will cut it
    tree_method="hist",
    random_state=random_state,
)

param_dist = {
    "learning_rate": np.linspace(0.02, 0.08, 13),
    "max_depth": list(range(3, 11)),
    "subsample": np.linspace(0.6, 1.0, 13),
    "colsample_bytree": np.linspace(0.6, 1.0, 13),
    "min_child_weight": [1, 2, 5, 10, 20, 30, 40],
    "gamma": [0.0, 0.25, 0.5, 1.0, 2.0],
    "reg_alpha": [0.0, 0.01, 0.1, 0.5, 1.0],
    "reg_lambda": [0.5, 1.0, 2.0, 5.0],
}

sampler = ParameterSampler(param_dist, n_iter=n_iter, random_state=random_state)

best = {
    "val_mae_clip": np.inf,
    "params": None,
    "model": None,
    "pred_min_val_clip": None,
    "pred_max_val_clip": None,
}

for i, params in enumerate(sampler, start=1):
    model = XGBRegressor(
        **base_params,
        **params,
        early_stopping_rounds=50,   # <-- optimization that preserves intent
        eval_metric="mae",
    )

    model.fit(
        Xtr, ytr,
        sample_weight=wtr,
        eval_set=[(Xva, yva)],
        verbose=False,
    )

    pred_val = model.predict(Xva)
    mae_val_clip, pmin, pmax, _ = evaluate_with_clip_0_100(yva, pred_val)

    if mae_val_clip < best["val_mae_clip"]:
        best.update(
            val_mae_clip=float(mae_val_clip),
            params=params,
            model=model,
            pred_min_val_clip=float(pmin),
            pred_max_val_clip=float(pmax),
        )

print("Best (by Val 2020 MAE with clip):")
print("  val_mae_clip:", f"{best['val_mae_clip']:.4f}")
print("  params:", best["params"])
print("  pred range val (clip):", f"[{best['pred_min_val_clip']:.4f}, {best['pred_max_val_clip']:.4f}]")

best_model = best["model"]

Best (by Val 2020 MAE with clip):
  val_mae_clip: 14.1195
  params: {'subsample': np.float64(0.8333333333333333), 'reg_lambda': 0.5, 'reg_alpha': 0.01, 'min_child_weight': 10, 'max_depth': 10, 'learning_rate': np.float64(0.03), 'gamma': 0.25, 'colsample_bytree': np.float64(0.9333333333333333)}
  pred range val (clip): [0.0000, 62.7752]


This was the lowest validation score observed across all searches. However, because this run did not include evaluation on Test 2021, it serves only as an exploratory signal rather than a candidate model. Subsequent refits using these parameters (reported below) confirmed that the improvement did not generalize.

In [317]:
new_row = {
    "model_name": "XGBRegressor",
    "objective": "reg:absoluteerror",
    "tag": "xgb_manual_expanded_abs_n30_clip",   
    "used_sample_weight": True,
    "mae_val_2020": 14.1195,
    "mae_test_2021": None,         
    "pred_min_test": None,       
    "pred_max_test": None,          
}
df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)

/tmp/ipykernel_1138/1227314983.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)


#### **3. Refit of the validation‑only best model (val + test)** 

To verify whether the promising validation‑only configuration from the manual 100‑iteration search would generalize beyond the 2020 validation set, we refit the model using the full training data and evaluated it on both Val 2020 and Test 2021.

In [301]:
best_params = {
    "subsample": 0.8333333333333333,
    "reg_lambda": 0.5,
    "reg_alpha": 0.01,
    "min_child_weight": 10,
    "max_depth": 10,
    "learning_rate": 0.03,
    "gamma": 0.25,
    "colsample_bytree": 0.9333333333333333,
}

Xtr = X_train.to_numpy(dtype=np.float32, copy=False)
Xva = X_val.to_numpy(dtype=np.float32, copy=False)
Xte = X_test.to_numpy(dtype=np.float32, copy=False)
ytr = y_train.to_numpy(dtype=np.float32, copy=False)
yva = y_val.to_numpy(dtype=np.float32, copy=False)
yte = y_test.to_numpy(dtype=np.float32, copy=False)
wtr = sample_weight_train.astype(np.float32, copy=False)

def mae_clip(y_true, y_pred):
    return mean_absolute_error(y_true, np.clip(y_pred, 0.0, 100.0))

model = XGBRegressor(
    objective="reg:absoluteerror",
    n_estimators=300,     
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
    **best_params,
)

model.fit(Xtr, ytr, sample_weight=wtr)

pred_val = model.predict(Xva)
pred_test = model.predict(Xte)

mae_val_clip = mae_clip(yva, pred_val)
mae_test_clip = mae_clip(yte, pred_test)

print(f"Val 2020 MAE (clip):  {mae_val_clip:.4f}")
print(f"Test 2021 MAE (clip): {mae_test_clip:.4f}")
print(f"Pred range test (clip): [{np.clip(pred_test,0,100).min():.4f}, {np.clip(pred_test,0,100).max():.4f}]")

baseline_huber_clip = 15.2000  # seu valor já medido
print(f"Baseline Huber-15 MAE (clip): {baseline_huber_clip:.4f}")
print("Beats baseline on Test (clip)?", mae_test_clip < baseline_huber_clip)
print("Gap to baseline (Test, clip):", mae_test_clip - baseline_huber_clip)

Val 2020 MAE (clip):  14.1831
Test 2021 MAE (clip): 15.4340
Pred range test (clip): [0.0000, 53.1278]
Baseline Huber-15 MAE (clip): 15.2000
Beats baseline on Test (clip)? False
Gap to baseline (Test, clip): 0.23396949768066477


Although the validation‑only search produced one of the lowest Val 2020 MAE values observed (14.1195), the refit using those hyperparameters did **not** translate into improved generalization. The Test 2021 MAE increased to **15.4340**, remaining above the baseline Huber‑15 model (15.2000). This confirms that the validation‑only improvement was not stable and did not carry over to the held‑out test set.

In [318]:
new_row = {
    "model_name": "XGBRegressor",
    "objective": "reg:absoluteerror",
    "tag": "xgb_manual_expanded_abs_n30_refit_clip",  # ajuste o nome se quiser
    "used_sample_weight": True,
    "mae_val_2020": float(mae_val_clip),
    "mae_test_2021": float(mae_test_clip),
    "pred_min_test": float(np.clip(pred_test, 0, 100).min()),
    "pred_max_test": float(np.clip(pred_test, 0, 100).max()),
}
df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
df_results = df_results.sort_values(by="mae_val_2020", ascending=True).reset_index(drop=True)

#### **4. High‑capacity refit with early stopping disabled**  
A refit with **n_estimators = 5000** (no early stopping):

In [304]:
model_es = XGBRegressor(
    objective="reg:absoluteerror",
    n_estimators=5000,          # alto, mas para cedo
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
    **best_params,
)

model_es.fit(
    X_train.to_numpy(np.float32, copy=False),
    y_train.to_numpy(np.float32, copy=False),
    sample_weight=sample_weight_train.astype(np.float32, copy=False),
    eval_set=[(X_val.to_numpy(np.float32, copy=False), y_val.to_numpy(np.float32, copy=False))],
    verbose=False,
)

pred_test = model_es.predict(X_test.to_numpy(np.float32, copy=False))
print("best_iteration:", getattr(model_es, "best_iteration", None))
print("Test 2021 MAE (clip):", mae_clip(y_test.to_numpy(np.float32, copy=False), pred_test))

best_iteration: None
Test 2021 MAE (clip): 15.4248685836792


A high‑capacity refit with 5000 trees (no early stopping) yielded a Test 2021 MAE of 15.4249, essentially identical to the standard refit. This confirms that increasing model capacity does not recover the validation‑only improvement and does not improve generalization.

In [319]:
new_row = {
    "model_name": "XGBRegressor",
    "objective": "reg:absoluteerror",
    "tag": "xgb_manual_expanded_abs_n30_refit_no_es_clip",
    "used_sample_weight": True,
    "mae_val_2020": None,  # este experimento é test-only
    "mae_test_2021": float(mae_clip(y_test.to_numpy(np.float32, copy=False), pred_test)),
    "pred_min_test": float(np.clip(pred_test, 0, 100).min()),
    "pred_max_test": float(np.clip(pred_test, 0, 100).max()),
}
df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
df_results = df_results.sort_values(by="mae_val_2020", ascending=True, na_position="last").reset_index(drop=True)

/tmp/ipykernel_1138/1690258031.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)


#### **5. Simplified model (reduced depth and larger min_child_weight)**  
A deliberately simpler variant:

In [306]:
params_simpler = dict(best_params)
params_simpler.update({"max_depth": 4, "min_child_weight": 50})

model_simple = XGBRegressor(
    objective="reg:absoluteerror",
    n_estimators=800,
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
    **params_simpler,
)

model_simple.fit(
    X_train.to_numpy(np.float32, copy=False),
    y_train.to_numpy(np.float32, copy=False),
    sample_weight=sample_weight_train.astype(np.float32, copy=False),
)

pred_test = model_simple.predict(X_test.to_numpy(np.float32, copy=False))
print("Test 2021 MAE (clip):", mae_clip(y_test.to_numpy(np.float32, copy=False), pred_test))


Test 2021 MAE (clip): 15.527663230895996


This confirmed that reducing model complexity also failed to improve performance.

In [320]:
new_row = {
    "model_name": "XGBRegressor",
    "objective": "reg:absoluteerror",
    "tag": "xgb_manual_expanded_abs_n30_refit_simplified_clip",
    "used_sample_weight": True,
    "mae_val_2020": None,  # este experimento é test-only
    "mae_test_2021": float(mae_clip(y_test.to_numpy(np.float32, copy=False), pred_test)),
    "pred_min_test": float(np.clip(pred_test, 0, 100).min()),
    "pred_max_test": float(np.clip(pred_test, 0, 100).max()),
}
df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
df_results = df_results.sort_values(
    by="mae_val_2020",
    ascending=True,
    na_position="last"
).reset_index(drop=True)

/tmp/ipykernel_1138/2507125951.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)


#### **6. High‑capacity refit with proper early stopping**  
Same best_params, but now with:

- `n_estimators=5000`  
- `early_stopping_rounds=50`  
- `eval_metric="mae"`

In [308]:
model_es = XGBRegressor(
    objective="reg:absoluteerror",
    n_estimators=5000,
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
    early_stopping_rounds=50,   # <-- no construtor
    eval_metric="mae",          # <-- no construtor
    **best_params,
)

model_es.fit(
    X_train.to_numpy(np.float32, copy=False),
    y_train.to_numpy(np.float32, copy=False),
    sample_weight=sample_weight_train.astype(np.float32, copy=False),
    eval_set=[(
        X_val.to_numpy(np.float32, copy=False),
        y_val.to_numpy(np.float32, copy=False)
    )],
    verbose=False,
)

pred_test = model_es.predict(X_test.to_numpy(np.float32, copy=False))
print("best_iteration:", getattr(model_es, "best_iteration", None))
print("Test 2021 MAE (clip):", mae_clip(y_test.to_numpy(np.float32, copy=False), pred_test))

best_iteration: 1152
Test 2021 MAE (clip): 15.413751602172852


Early stopping improved stability relative to the no‑ES version, but still remained well above the baseline.

In [322]:
new_row = {
    "model_name": "XGBRegressor",
    "objective": "reg:absoluteerror",
    "tag": "xgb_manual_expanded_abs_n30_refit_es50_clip",
    "used_sample_weight": True,
    "mae_val_2020": None,  # este experimento é test-only
    "mae_test_2021": float(mae_clip(y_test.to_numpy(np.float32, copy=False), pred_test)),
    "pred_min_test": float(np.clip(pred_test, 0, 100).min()),
    "pred_max_test": float(np.clip(pred_test, 0, 100).max()),
}

df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)

df_results = df_results.sort_values(
    by="mae_val_2020",
    ascending=True,
    na_position="last"
).reset_index(drop=True)

/tmp/ipykernel_1138/3440800954.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)


In [330]:
df_results = df_results.drop_duplicates().reset_index(drop=True)
df_results = (
    df_results
    .drop_duplicates(subset=["tag"], keep="first")
    .reset_index(drop=True)
)
df_results

,model_name,objective,tag,used_sample_weight,mae_val_2020,mae_test_2021,pred_min_test,pred_max_test
0,XGBRegressor,reg:absoluteerror,xgb_manual_expanded_abs_n30_clip,True,14.1195,NaN,NaN,NaN
1,XGBRegressor,reg:absoluteerror,xgb_tuned_expanded_abs_n30_clip,True,14.1227,15.3649,0.0000,54.3557
2,XGBRegressor,reg:absoluteerror,xgb_tuned_absoluteerror_clip,True,14.1274,15.5146,0.0000,55.2573
3,XGBRegressor,reg:absoluteerror,xgb_tuned_reg:absoluteerror,True,14.1279,15.5150,-3.2848,55.2573
4,XGBRegressor,reg:absoluteerror,xgb_tuned_expanded_reg:absoluteerror,True,14.1280,15.2541,0.0000,55.0586
5,XGBRegressor,reg:pseudohubererror,xgb_tuned_expanded_reg:pseudohubererror,True,14.1431,15.3699,0.0000,54.9421
6,XGBRegressor,reg:pseudohubererror,xgb_tuned_pseudohuber_clip,True,14.1455,15.3036,0.0000,58.3352
7,XGBRegressor,reg:pseudohubererror,xgb_tuned_reg:pseudohubererror,True,14.1467,15.3044,-3.2895,58.3352
8,XGBRegressor,reg:squarederror,xgb_tuned_expanded_reg:squarederror,True,14.1685,15.4683,0.0000,52.6271
9,XGBRegressor,reg:squarederror,xgb_tuned_squarederror_clip,True,14.1788,15.4563,0.0000,54.7270


### **4.6.6 Final Assessment: No Challenger Surpassed the Baseline**

Across:

- linear models  
- GLMs  
- hurdle models  
- XGBoost point‑runs  
- tuned models  
- expanded tuned models  
- 100‑iteration searches  
- early stopping  
- clipping  

…the conclusion was consistent:

> **No model surpassed the Huber‑15 baseline on Test 2021.**

The best challenger reached **15.2541**, but the baseline remained superior at **15.2000**.

### **4.6.7 Final Model Selection**

Given:

- superior generalization under drift  
- stability  
- interpretability  
- simplicity of deployment  
- and the fact that no challenger beat it  

The **Huber‑15 regression model** was selected as the final model for implementation.

# 5. Evaluation & Persistence (Cycle 3)

## **5.1 Select the winner (baseline vs best XGB)**

The goal of this step is to formalize the model‑selection decision using a simple and reproducible rule. All candidate models from Cycle 3 are stored in `df_results`, including the baseline (Huber‑15) and the best XGBoost variants obtained during tuning.

The selection procedure follows two principles:

1. **Ranking by the predefined selection metric**  
   Models are first ordered by their **Val 2020 MAE (clip)**, which was the metric used throughout Cycle 3 for hyperparameter search.

2. **Final decision based on generalization**  
   Even if a model ranks first on validation, it is only declared the champion if it also **beats the baseline on Test 2021**.  
   This ensures that the final choice prioritizes out‑of‑sample performance and avoids overfitting to the validation split.

The following cell implements this rule and prints the full comparison:

In [331]:
# --- Dual-mode selection: clip and no-clip in one run ---

SEL_COL  = "mae_val_2020"
TEST_COL = "mae_test_2021"

MODES = {
    "clip": {
        "filter": lambda df: df["tag"].astype(str).str.contains("clip"),
        "baseline": "baseline_huber15_clip",
    },
    "no_clip": {
        "filter": lambda df: ~df["tag"].astype(str).str.contains("clip"),
        "baseline": "baseline_huber15",
    },
}

# --- Run both modes ---
results = [evaluate_mode(m, df_results) for m in MODES]

# --- Pretty print ---
for r in results:
    print(f"\n=== Mode: {r['mode']} ===")
    print(f"Baseline:        {r['baseline_tag']}  (Test={r['baseline_test']:.4f})")
    print(f"Selected by Val: {r['best_tag']}      (Test={r['best_test']:.4f})")
    print(f"Gap to baseline: {r['gap']:+.4f}")
    print(f"Champion:        {r['champion']}")


=== Mode: clip ===
Baseline:        baseline_huber15_clip  (Test=15.2000)
Selected by Val: xgb_tuned_expanded_abs_n30_clip      (Test=15.3649)
Gap to baseline: +0.1649
Champion:        baseline_huber15_clip

=== Mode: no_clip ===
Baseline:        baseline_huber15  (Test=15.2127)
Selected by Val: xgb_tuned_reg:absoluteerror      (Test=15.5150)
Gap to baseline: +0.3023
Champion:        baseline_huber15


In [332]:
df_results

,model_name,objective,tag,used_sample_weight,mae_val_2020,mae_test_2021,pred_min_test,pred_max_test
0,XGBRegressor,reg:absoluteerror,xgb_manual_expanded_abs_n30_clip,True,14.1195,NaN,NaN,NaN
1,XGBRegressor,reg:absoluteerror,xgb_tuned_expanded_abs_n30_clip,True,14.1227,15.3649,0.0000,54.3557
2,XGBRegressor,reg:absoluteerror,xgb_tuned_absoluteerror_clip,True,14.1274,15.5146,0.0000,55.2573
3,XGBRegressor,reg:absoluteerror,xgb_tuned_reg:absoluteerror,True,14.1279,15.5150,-3.2848,55.2573
4,XGBRegressor,reg:absoluteerror,xgb_tuned_expanded_reg:absoluteerror,True,14.1280,15.2541,0.0000,55.0586
5,XGBRegressor,reg:pseudohubererror,xgb_tuned_expanded_reg:pseudohubererror,True,14.1431,15.3699,0.0000,54.9421
6,XGBRegressor,reg:pseudohubererror,xgb_tuned_pseudohuber_clip,True,14.1455,15.3036,0.0000,58.3352
7,XGBRegressor,reg:pseudohubererror,xgb_tuned_reg:pseudohubererror,True,14.1467,15.3044,-3.2895,58.3352
8,XGBRegressor,reg:squarederror,xgb_tuned_expanded_reg:squarederror,True,14.1685,15.4683,0.0000,52.6271
9,XGBRegressor,reg:squarederror,xgb_tuned_squarederror_clip,True,14.1788,15.4563,0.0000,54.7270


In [347]:
df_results.to_json()

'{"model_name":{"0":"XGBRegressor","1":"XGBRegressor","2":"XGBRegressor","3":"XGBRegressor","4":"XGBRegressor","5":"XGBRegressor","6":"XGBRegressor","7":"XGBRegressor","8":"XGBRegressor","9":"XGBRegressor","10":"XGBRegressor","11":"XGBRegressor","12":"XGBRegressor","13":"XGBRegressor","14":"XGBRegressor","15":"XGBRegressor","16":"XGBRegressor","17":"XGBRegressor","18":"TweedieRegressor","19":"TweedieRegressor","20":"TweedieRegressor","21":"TweedieRegressor","22":"HuberRegressor","23":"HuberRegressor","24":"Hurdle(LogReg + Huber)","25":"XGBRegressor","26":"XGBRegressor","27":"XGBRegressor"},"objective":{"0":"reg:absoluteerror","1":"reg:absoluteerror","2":"reg:absoluteerror","3":"reg:absoluteerror","4":"reg:absoluteerror","5":"reg:pseudohubererror","6":"reg:pseudohubererror","7":"reg:pseudohubererror","8":"reg:squarederror","9":"reg:squarederror","10":"reg:squarederror","11":"reg:absoluteerror","12":"reg:absoluteerror","13":"reg:absoluteerror","14":"reg:squarederror","15":"reg:squarederr

1. **Main conclusion**

   
   Despite extensive tuning and several XGBoost variants achieving strong validation performance (Val 2020 MAE ≈ 14.12–14.18), none of them outperformed the Huber-15 baseline on the held-out Test 2021 set, in either **no-clip** or **clip (0–100)** mode.

2. **Interpretation**

   
   This Val–Test contrast suggests that the improvements observed on 2020 do not carry over to 2021 under the chosen temporal split (potentially due to concept drift between 2020 and 2021), so validation gains alone are not sufficient to justify replacing the baseline.

3. **Selection vs decision rule**

   
   Within each evaluation mode, models are **selected by Val 2020 MAE** and the final “champion” decision is made by **Test 2021 MAE** under the same mode; under this rule, **Huber-15 remains the champion**.

4. **Short “punchline”**

   
   In this setting, the most complex model was not the most reliable out-of-sample; the robust Huber-15 baseline delivered the best generalization on Test 2021.


```mermaid
flowchart TD
    A["Goal: choose the Cycle 3 champion<br/>under the frozen protocol (Huber-15 input space)"] --> B["Protocol invariants (guardrails)<br/>• Temporal split: train ≤2019, val=2020, test=2021<br/>• 15 numeric columns only<br/>• Median imputer fit on train only<br/>• Recency weights (λ=0.05) on train only"]

    B --> C["Candidates evaluated"]
    C --> H["Baseline: HuberRegressor (Huber-15)"]
    C --> X["Challengers: XGBoost variants<br/>• point runs (3 objectives)<br/>• tuned runs (PredefinedSplit train/val)<br/>• expanded tuning (reg_alpha, reg_lambda, gamma, min_child_weight, ...)"]
    C --> T["Other probes<br/>• TweedieRegressor (powers)<br/>• Minimal hurdle (LogReg + Huber)"]

    %% Selection / Decision
    H --> S["Selection metric (for tuning):<br/>Val 2020 MAE (within mode)"]
    X --> S
    T --> S

    S --> D["Decision metric (final):<br/>Test 2021 MAE (within mode)"]

    %% What happened
    D --> V1["Observed pattern:<br/>XGBoost wins on Val 2020 (≈14.12–14.18)<br/>but loses on Test 2021 (≥15.25–15.89)"]
    D --> V2["Huber-15 remains best on Test 2021<br/>• no-clip: 15.2127<br/>• clip 0–100: 15.2000"]

    %% Why Huber wins
    V1 --> W["Key reason for Huber-15 win:<br/>Generalization under temporal drift"]
    W --> W1["2021 distribution shifts (more zeros / drift)<br/>→ validation gains (2020) don't transfer"]
    W --> W2["Huber loss is robust to outliers / heavy tails<br/>→ fewer catastrophic errors in 2021"]
    W --> W3["Small, fixed feature space (15 numeric)<br/>→ reduced variance vs tuned tree ensembles"]

    %% Supporting signals
    V1 --> Z["Supporting signals observed"]
    Z --> Z1["Tuned XGB shows lower Val MAE but higher Test MAE<br/>→ over-specialization to 2020 patterns"]
    Z --> Z2["Pseudo-Huber objective had unstable ranges in point run<br/>→ risk of extreme predictions"]
    Z --> Z3["Hurdle probe degraded MAE (error propagation)<br/>→ not beneficial in minimal form"]
    Z --> Z4["Clip helps Huber slightly (~-0.013 MAE) but<br/>doesn't change winner"]

    %% Champion decision
    V2 --> CH["Champion decision (per mode):<br/>Champion = argmin(Test 2021 MAE)"]
    CH --> OUT["Winner: Huber-15 baseline<br/>Most stable and best generalization on 2021"]

    %% Visual emphasis
    classDef winner fill:#dff7df,stroke:#2e7d32,stroke-width:2px;
    classDef warn fill:#fff3cd,stroke:#b26a00,stroke-width:1px;
    class H,OUT,V2,CH winner;
    class V1,Z,W warn;


```mermaid
flowchart LR
    A["Same protocol for everyone<br/>(same split, same 15 features, same preprocessing)"] --> B["Compare two signals"]
    B --> C["Validation (2020)<br/>(Looks good now)"]
    B --> D["Test (2021)<br/>(will it still work next year?)"]

    C --> XGB["XGBoost: very low Val MAE<br/>(learns many fine patterns)"]
    D --> XGBT["But higher Test MAE in 2021<br/>(patterns don't transfer under drift)"]

    C --> HUB["Huber-15: Val MAE not the best<br/>(simpler fit)"]
    D --> HUBT["But best Test MAE in 2021<br/>(stable under drift + robust loss)"]

    XGB --> E["Reason: high flexibility → can overfit 2020 quirks"]
    E --> XGBT

    HUB --> F["Reason: robust loss + low-variance model<br/>→ avoids big mistakes when 2021 shifts"]
    F --> HUBT

    XGBT --> G["Champion rule:<br/>pick lowest Test MAE (generalization)"]
    HUBT --> G

    G --> WIN["Winner: Huber-15<br/>(most consistent next-year performance)"]

    classDef win fill:#dff7df,stroke:#2e7d32,stroke-width:2px;
    classDef lose fill:#ffe6e6,stroke:#b71c1c,stroke-width:1px;
    class WIN,HUBT win;
    class XGBT lose;


## 5.2 — Persist the champion model (`champion.joblib`)

The Cycle 3 champion is the **Huber-15 baseline** under the frozen protocol (15 numeric columns + train-only median imputation + train-only recency weights, λ=0.05).  
Since no challenger improved **Test 2021 MAE** under the same guardrails, we persist this fitted model as the official Cycle 3 artifact.

- **Artifact:** `models/cycle_03/champion.joblib`
- **Model:** `HuberRegressor`
- **Protocol:** `Baseline_Huber15_recency0p05_medfill`


In [346]:
# --- 5.2 Persist the champion model ---


# Ensure output dir exists
CYCLE3_MODELS_DIR = PROJECT_ROOT / "models" / "cycle_03"
CYCLE3_MODELS_DIR.mkdir(parents=True, exist_ok=True)

CHAMPION_PATH = CYCLE3_MODELS_DIR / "champion.joblib"

# Champion = Huber baseline under the frozen protocol
champion_model = huber  # assumes you already fit() it in 3.4

joblib.dump(champion_model, CHAMPION_PATH)
print("Saved champion model:", CHAMPION_PATH)

Saved champion model: /mnt/c/Users/Daniel/OneDrive/Documentos/_Cursos/Outros/PopForecast/models/cycle_03/champion.joblib


## 5.3 — Record run metadata (traceability bundle)

To make the Cycle 3 outcome auditable and reproducible, we write a metadata JSON capturing:
- **protocol guardrails** (temporal split, 15-column list, imputation scope, recency settings, NaN-year policy),
- **champion identity and params**,
- **metrics** reported in two modes:
  - **no_clip** (decision metric)
  - **clip_0_100** (evaluation-only sanity check)
- **artifact hashes** (SHA256) and key environment versions.

- **Run metadata:** `models/cycle_03/run_metadata_cycle3.json`
- **Decision metric:** `mae_test_2021_no_clip`
- **Reported modes:** `no_clip`, `clip_0_100` (clip is evaluation-only)


In [345]:
# --- 5.3 Record run metadata (audit) ---
    
# baseline rows
base_no_clip = df_results.loc[df_results["tag"].eq("baseline_huber15"), "mae_test_2021"].iloc[0]
base_clip    = df_results.loc[df_results["tag"].eq("baseline_huber15_clip"), "mae_test_2021"].iloc[0]

# candidates
cand = df_results[df_results["model_name"] != "HuberRegressor"].copy()
cand["is_clip"] = cand["tag"].map(_is_clip_tag)

best_cand_no_clip = cand.loc[~cand["is_clip"], "mae_test_2021"].min()
best_cand_clip    = cand.loc[cand["is_clip"],  "mae_test_2021"].min()

summary = {
    "baseline_beaten_any_no_clip": bool(best_cand_no_clip < base_no_clip) if pd.notna(best_cand_no_clip) else None,
    "baseline_beaten_any_clip":    bool(best_cand_clip    < base_clip)    if pd.notna(best_cand_clip)    else None,
    "best_candidate_no_clip": float(best_cand_no_clip) if pd.notna(best_cand_no_clip) else None,
    "best_candidate_clip":    float(best_cand_clip)    if pd.notna(best_cand_clip)    else None,
}
# print(f'summary: {summary}')


# Governance + baseline audit files (already loaded earlier)
governance_path = CYCLE2_MODELS_DIR / "frozen_config_cycle2.json"
baseline_audit_v3_path = CYCLE2_MODELS_DIR / "baseline_huber15_audit_v3_from_pack.json"

metadata = {
    "cycle": 3,
    "timestamp_utc": pd.Timestamp.utcnow().isoformat(),
    "champion": {
        "tag": "baseline_huber15",              # canonical champion id (no-clip)
        "model_name": "HuberRegressor",
        "protocol_name": protocol_key,
        "params": huber.get_params(),
    
        # clarify how winner is decided
        "selection": {
            "decision_metric": "mae_test_2021_no_clip",   # winner decided on no-clip
            "reported_modes": ["no_clip", "clip_0_100"],
            "clip_is_eval_only": True,
        },
    
        # report both, with explicit naming
        "metrics": {
            "val_2020_mae_no_clip": float(mae_val),
            "test_2021_mae_no_clip": float(mae_test),
    
            # if you already computed these, store them; otherwise compute here
            "val_2020_mae_clip_0_100": float(mean_absolute_error(
                y_val.to_numpy(float),
                np.clip(y_pred_val, 0.0, 100.0),
            )),
            "test_2021_mae_clip_0_100": float(mean_absolute_error(
                y_test.to_numpy(float),
                np.clip(y_pred_test, 0.0, 100.0),
            )),
        },
    
        # keep raw prediction range (no-clip preds)
        "pred_range_test_no_clip": {
            "min": float(np.min(y_pred_test)),
            "max": float(np.max(y_pred_test)),
        },
    },
    "protocol_guardrails": {
        "split": governance_cfg["decision_split"],
        "imputation": governance_cfg["baseline_protocols"][protocol_key]["preprocessing"]["imputation"],
        "recency_weighting": governance_cfg["baseline_protocols"][protocol_key]["recency_weighting"],
        "numeric_cols_15": numeric_cols,
        "nan_year_policy": "train",
    },
    "artifacts": {
        "champion_joblib": str(CHAMPION_PATH),
        "champion_joblib_sha256": _sha256_file(CHAMPION_PATH),
        "governance_config_path": str(governance_path),
        "governance_config_sha256": _sha256_file(governance_path) if governance_path.exists() else None,
        "baseline_audit_v3_path": str(baseline_audit_v3_path),
        "baseline_audit_v3_sha256": _sha256_file(baseline_audit_v3_path) if baseline_audit_v3_path.exists() else None,
    },
    "environment": {
        "python": sys.version,
        "platform": platform.platform(),
        "numpy": np.__version__,
        "pandas": pd.__version__,
        "sklearn": sklearn.__version__,
        "xgboost": xgb.__version__,
    },
    "summary": {
        "baseline_beaten_by_any_candidate": bool((df_results["mae_test_2021"] < mae_test).any())
        if "df_results" in globals() and "mae_test_2021" in df_results.columns
        else None
    },
}

metadata["summary"] = summary

RUN_METADATA_PATH.write_text(json.dumps(metadata, indent=2), encoding="utf-8")
print("Saved run metadata:", RUN_METADATA_PATH)


summary: {'baseline_beaten_any_no_clip': False, 'baseline_beaten_any_clip': False, 'best_candidate_no_clip': 15.254113425308288, 'best_candidate_clip': 15.303604679397173}
Saved run metadata: /mnt/c/Users/Daniel/OneDrive/Documentos/_Cursos/Outros/PopForecast/models/cycle_03/run_metadata_cycle3.json


# 6. Cycle 3 — Final outcome 

Across point-runs, tuning (PredefinedSplit train/val), and expanded searches, XGBoost candidates achieved strong validation MAE but did not translate that advantage into **Test 2021** improvements.  
Under the frozen protocol and the temporal decision split, the **Huber-15** baseline remained the most reliable generalizer and is therefore the Cycle 3 champion.

Next steps:
- Use `champion.joblib` as the reference model for subsequent cycles.
- Treat clip(0–100) as a reporting sanity check only; keep all model selection decisions based on **no_clip** test MAE.
